In [2]:
# external modules
import sys
import random
import numpy as np
import pandas as pd
import collections
import matplotlib.pyplot as plt
import japanize_matplotlib
from matplotlib.dates import DateFormatter
import seaborn as sns
import plotly.graph_objects as go
import statsmodels.api as sm

# Pandas setting
# Display 6 columns for viewing purposes
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

# Reduce decimal points to 2
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# read temporary processed data
df = pd.read_csv("./../data/processed/tse1_unitchanged.csv", index_col=[0, 1, 2])
df_2008 = pd.read_csv("../data/processed/tse1_2008_balanced.csv", index_col=[0, 1, 2])

In [220]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], :].isna().mean()

決算期（年）                      0.00
決算期（月）                      0.00
決算期                         0.00
連結基準フラグ                     0.00
決算種別フラグ                     0.00
決算発表日                       0.00
事業年度開始年月日［３ヵ月］              0.01
事業年度終了年月日                   0.00
１株当たり利益［３ヵ月］                0.03
１株当たり利益［累計］                 0.01
当期純利益（連結）［累計］               0.19
【ＱＴＲ】当期利益                   0.00
棚卸資産                        0.02
受取手形・売掛金／売掛金及びその他の短期債権      0.01
資本的支出                       0.00
売上総利益［累計］                   0.00
【ＱＴＲ】売上総利益                  0.00
販売費及び一般管理費［累計］              0.01
法人税等［累計］                    0.00
法人税・住民税及び事業税合計／当期税金費用［累計］   0.27
税金等調整前当期純利益［累計］             0.00
税金等調整前当期利益［累計］              0.04
【ＱＴＲ】税金等調整前当期利益             0.04
１株当たり税引前利益                  0.01
売上高・営業収益［累計］                0.00
【ＱＴＲ】売上高・営業収益               0.01
１株当たり売上高                    0.01
期末従業員数                      0.00
純資産合計／資本合計                  0.01
【ＱＴＲ】純資産                    0.01
１株当たり純資産  

## 欠損処理

* 資本的支出と期末従業員数

四半期データは2008年に四半期開始、2011に簡素化されてる。(簡素化後、3ヶ月は任意。累積は義務)  
https://www.asb.or.jp/jp/accounting_standards/accounting_standards/y2007/2007-0314.html  
https://www.asb.or.jp/jp/accounting_standards/accounting_standards/y2011/2011-0325.html  
簡素化に伴い、資本的支出(CF計算書)、期末従業員数の報告が四半期で報告されなくなる。
--> Zhang et al. (2004)に倣い、資本的支出は4等分、期末従業員数は一定とする。

In [4]:
# proportion of missing value
df_2008[["資本的支出", "期末従業員数"]].isna().mean()

資本的支出    0.30
期末従業員数   0.54
dtype: float64

In [5]:
df_2008[["資本的支出", "期末従業員数"]].describe()

,資本的支出,期末従業員数
count,"51,955.00","34,556.00"
mean,"18,533,372,129.73","9,439.12"
std,"108,863,616,446.61","30,128.37"
min,"1,000,000.00",13.00
25%,"411,000,000.00",730.00
50%,"1,639,000,000.00","1,849.00"
75%,"6,891,000,000.00","5,690.25"
max,"5,121,960,000,000.00","385,243.00"


* 期末従業員数

欠損は前四半期時点の従業員数を代入する。(従業員数は変化していないという仮定)

In [6]:
df_2008.loc[pd.IndexSlice[:, 2008, 6], :]["期末従業員数"].isna().sum()

52

In [7]:
df_2008.loc[pd.IndexSlice[:, 2008, 6], :]["期末従業員数"][
    df_2008.loc[pd.IndexSlice[:, 2008, 6], :]["期末従業員数"].isna()
]

企業名              決算期（年）  決算期（月）
ひらまつ             2008    6        nan
アルインコ            2008    6        nan
エムティーアイ          2008    6        nan
カワチ薬品            2008    6        nan
キャリアデザインセンター     2008    6        nan
キーエンス            2008    6        nan
グリムス             2008    6        nan
コナカ              2008    6        nan
サイバーエージェント       2008    6        nan
システムソフト          2008    6        nan
シミックホールディングス     2008    6        nan
ジャパンベストレスキューシステ  2008    6        nan
ステップ             2008    6        nan
ティア              2008    6        nan
ディア・ライフ          2008    6        nan
ネクシィーズグループ       2008    6        nan
パラカ              2008    6        nan
ファルコホールディングス     2008    6        nan
フルキャストホールディングス   2008    6        nan
ホソカワミクロン         2008    6        nan
メディカルシステムネットワーク  2008    6        nan
ヤマウラ             2008    6        nan
ヤマト              2008    6        nan
ユニカフェ            2008    6        nan
ヨコレイ             2008    6        nan
ランドビジネス          2

最初の期2008Q1が欠損してしまっている企業が 52 / 1436 ある。df_2008だけでは欠損値補填ができないのでdfに戻ってさらにさかのぼって補填

In [8]:
year_range = np.arange(2008, 2021)
month_range = [6, 9, 12, 3]

# check if firm recorded in all quater
# start from firm recorded in 2008Q1
recorded_firm_from_2008 = set(df.loc[pd.IndexSlice[:, 2008, 6], :].index.get_level_values("企業名").unique())

for i in year_range:
    for j in month_range:
        if j == 3: # if Mar, +1 year
            i += 1
        recorded_firm_from_2008 = recorded_firm_from_2008 & set(df.loc[pd.IndexSlice[:, i, j], :].index.get_level_values("企業名").unique())

df_2008 = df.loc[pd.IndexSlice[recorded_firm_from_2008, :, :], :] # 遡るためrecorded_firm_from_2008の企業の全期間 : のデータフレームをもう一度作成

In [9]:
df_2008.loc[pd.IndexSlice[["アサヒグループホールディングス", "アシックス"], :, :], :]["期末従業員数"].iloc[70:90]

企業名              決算期（年）  決算期（月）
アサヒグループホールディングス  2018    6              nan
                         9              nan
                         12       28,055.00
                 2019    3              nan
                         6              nan
                         9              nan
                         12       29,327.00
                 2020    3              nan
                         6              nan
                         9              nan
                         12       29,850.00
                 2021    3              nan
アシックス            1999    9              nan
                 2000    3         4,270.00
                         9         4,348.00
                 2001    3         4,273.00
                         9         4,233.00
                 2002    3         4,109.00
                         9         4,236.00
                 2003    3         4,132.00
Name: 期末従業員数, dtype: float64

In [10]:
# forward fill groupby firm level
df_2008["期末従業員数"] = df_2008.groupby(level=0)["期末従業員数"].fillna(method='ffill')

<ipython-input-10-4ee0692f63b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2008["期末従業員数"] = df_2008.groupby(level=0)["期末従業員数"].fillna(method='ffill')


In [11]:
df_2008.loc[pd.IndexSlice[["アサヒグループホールディングス", "アシックス"], :, :], :]["期末従業員数"].iloc[70:90]

企業名              決算期（年）  決算期（月）
アサヒグループホールディングス  2018    6        30,864.00
                         9        30,864.00
                         12       28,055.00
                 2019    3        28,055.00
                         6        28,055.00
                         9        28,055.00
                         12       29,327.00
                 2020    3        29,327.00
                         6        29,327.00
                         9        29,327.00
                         12       29,850.00
                 2021    3        29,850.00
アシックス            1999    9              nan
                 2000    3         4,270.00
                         9         4,348.00
                 2001    3         4,273.00
                         9         4,233.00
                 2002    3         4,109.00
                         9         4,236.00
                 2003    3         4,132.00
Name: 期末従業員数, dtype: float64

* 資本的支出

欠損はほとんど半期報告のため。  
半期の値を2で割って前四半期に割り当てる。  
1年報告しかしていないなら4で割って各四半期に割り当てる。

まずは資本的支出の欠損パターンを確認

In [12]:
# check missing patterns
df_2008_capxisna = df_2008.loc[pd.IndexSlice[:, 2008:, 3:], :][
    df_2008.loc[pd.IndexSlice[:, 2008:, 3:], :]["資本的支出"].isna()
]

df_2008_capxisna.index.get_level_values("決算期（月）").unique()

Int64Index([9, 3, 6, 12], dtype='int64', name='決算期（月）')

どの四半期も欠損し得る

In [13]:
df_2008_capxisna["資本的支出"]

企業名          決算期（年）  決算期（月）
あい　ホールディングス  2011    9        nan
             2012    3        nan
                     9        nan
             2013    3        nan
                     9        nan
                               ..
ｅＢＡＳＥ        2018    12       nan
             2019    6        nan
                     12       nan
             2020    6        nan
                     12       nan
Name: 資本的支出, Length: 22747, dtype: float64

In [14]:
# firm missing Q1
missQ1 = df_2008_capxisna.loc[pd.IndexSlice[:, :, [6]], :]["資本的支出"].index.get_level_values(0).unique()
missQ1

Index(['あみやき亭', 'あらた', 'いすゞ自動車', 'いなげや', 'きんでん', 'ぐるなび', 'さくらインターネット', 'たけびし',
       'なとり', 'はせがわ',
       ...
       'ＴＯＹＯ　ＴＩＲＥ', 'ＴＰＲ', 'ＴＹＫ', 'ＵＡＣＪ', 'ＵＴグループ', 'ＶＴホールディングス',
       'ＷＤＢホールディングス', 'ＷＯＷＯＷ', 'ＺＯＺＯ', 'ｅＢＡＳＥ'],
      dtype='object', name='企業名', length=1082)

In [15]:
# firm missing Q2
missQ2 = df_2008_capxisna.loc[pd.IndexSlice[:, :, [9]], :]["資本的支出"].index.get_level_values(0).unique()
missQ2

Index(['あい　ホールディングス', 'いであ', 'きちりホールディングス', 'ひらまつ', 'アイティメディア', 'アイフィスジャパン',
       'アイ・エス・ビー', 'アイ・オー・データ機器', 'アウトソーシング', 'アグロ　カネショウ',
       ...
       'ＧＭＯペパボ', 'Ｉ－ＰＥＸ', 'ＩＮＰＥＸ', 'ＪＵＫＩ', 'ＫＮＴ－ＣＴホールディングス', 'ＭｏｎｏｔａＲＯ', 'ＮＪＳ',
       'ＳＢＳホールディングス', 'ＴＨＫ', 'ＴＯＹＯ　ＴＩＲＥ'],
      dtype='object', name='企業名', length=194)

In [16]:
# firm missing Q3
missQ3 = df_2008_capxisna.loc[pd.IndexSlice[:, :, [12]], :]["資本的支出"].index.get_level_values(0).unique()
missQ3

Index(['あみやき亭', 'あらた', 'いすゞ自動車', 'いなげや', 'きんでん', 'ぐるなび', 'さくらインターネット', 'たけびし',
       'なとり', 'はせがわ',
       ...
       'ＴＯＹＯ　ＴＩＲＥ', 'ＴＰＲ', 'ＴＹＫ', 'ＵＡＣＪ', 'ＵＴグループ', 'ＶＴホールディングス',
       'ＷＤＢホールディングス', 'ＷＯＷＯＷ', 'ＺＯＺＯ', 'ｅＢＡＳＥ'],
      dtype='object', name='企業名', length=1074)

In [17]:
# firm missing Q4
missQ4 = df_2008_capxisna.loc[pd.IndexSlice[:, :, [3]], :]["資本的支出"].index.get_level_values(0).unique()
missQ4

Index(['あい　ホールディングス', 'いであ', 'きちりホールディングス', 'アイフィスジャパン', 'アイ・エス・ビー',
       'アイ・オー・データ機器', 'アウトソーシング', 'アグロ　カネショウ', 'アゴーラ　ホスピタリティー　グ',
       'アサヒグループホールディングス',
       ...
       'ＧＭＯペパボ', 'Ｉ－ＰＥＸ', 'ＩＮＰＥＸ', 'ＪＵＫＩ', 'ＫＮＴ－ＣＴホールディングス', 'ＭｏｎｏｔａＲＯ', 'ＮＪＳ',
       'ＳＢＳホールディングス', 'ＴＨＫ', 'ＴＯＹＯ　ＴＩＲＥ'],
      dtype='object', name='企業名', length=194)

In [18]:
missQ1 & missQ2 & missQ3 & missQ4

Index(['アシックス', 'インテージホールディングス', 'エス・サイエンス', 'オムロン', 'クラレ', 'コナミホールディングス',
       'サイバネットシステム', 'サイバーコム', 'サカタインクス', 'システムソフト', 'ソディック', 'ソリトンシステムズ',
       'デジタルガレージ', 'トラスコ中山', 'フルキャストホールディングス', 'マキタ', 'メック', 'ユニカフェ',
       'ユニ・チャーム', 'ローランド　ディー．ジー．', 'ヴィンクス', '井関農機', '富士ソフト', '小林製薬', '市光工業',
       '手間いらず', '日本ペイントホールディングス', '日本商業開発', '日機装', '日清紡ホールディングス', '日華化学',
       '星光ＰＭＣ', '東京応化工業', '東洋インキＳＣホールディングス', '江崎グリコ', '資生堂', '電通国際情報サービス',
       '鳥居薬品', 'ＩＮＰＥＸ', 'ＫＮＴ－ＣＴホールディングス', 'ＴＨＫ', 'ＴＯＹＯ　ＴＩＲＥ'],
      dtype='object', name='企業名')

In [19]:
df_2008.loc[pd.IndexSlice["日本商業開発", :, :], :]["資本的支出"]

企業名     決算期（年）  決算期（月）
日本商業開発  2003    3                   nan
        2004    3                   nan
        2005    3                   nan
        2006    3        107,000,000.00
        2007    3        150,000,000.00
                9                   nan
                12                  nan
        2008    3                   nan
                6                   nan
                9         52,000,000.00
                12        53,000,000.00
        2009    3         53,000,000.00
                6                   nan
                9                   nan
                12                  nan
        2010    3         10,000,000.00
                6         20,000,000.00
                9         20,000,000.00
                12        20,000,000.00
        2011    3         20,000,000.00
                6                   nan
                9                   nan
                12                  nan
        2012    3                   nan
                6

めんどくせぇ、でもどちらにせよ後の発表された資本的支出の額に前の欠損値分の資本的支出が含まれていて、欠損の分を問う分割して配分すればおっけー。

全部欠損している企業はもうどうしようもない、サンプルから落とすしかない。

In [20]:
# 欠損 50% 以上
len(df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=0)[
    df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=0) >= 0.5
])

7

In [21]:
# 欠損 40% 以上
len(df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=0)[
    df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=0) >= 0.4
])

16

In [22]:
# 欠損 30% 以上
len(df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=0)[
    df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=0) >= 0.3
])

1082

In [23]:
df_2008.loc[pd.IndexSlice["システムソフト", 2008:, :], :]["資本的支出"]

企業名      決算期（年）  決算期（月）
システムソフト  2008    3          1,000,000.00
                 6          3,000,000.00
                 9          5,000,000.00
                 12                  nan
         2009    3                   nan
                 6                   nan
                 9                   nan
                 12        12,000,000.00
         2010    3         12,000,000.00
                 6         12,000,000.00
                 9         12,000,000.00
                 12                  nan
         2011    3                   nan
                 6                   nan
                 9        944,000,000.00
                 12                  nan
         2012    3          1,000,000.00
                 6                   nan
                 9         21,000,000.00
                 12                  nan
         2013    3         26,000,000.00
                 6                   nan
                 9         54,000,000.00
                 12              

欠損率が大きくても、これなら埋めれる。(会計年度内最低1回報告している)

In [24]:
df_2008.loc[pd.IndexSlice["マキタ", 2008:, :], :]["資本的支出"]

企業名  決算期（年）  決算期（月）
マキタ  2008    3                      nan
             6                      nan
             9                      nan
             12                     nan
     2009    3                      nan
             6                      nan
             9                      nan
             12                     nan
     2010    3                      nan
             6                      nan
             9                      nan
             12                     nan
     2011    3                      nan
             6                      nan
             9                      nan
             12                     nan
     2012    3                      nan
             6                      nan
             9                      nan
             12                     nan
     2013    3                      nan
             6                      nan
             9                      nan
             12                     nan
     2014    3      

このような欠損(１年に渡って欠損している)は救いようがない

チェックすべき(除外すべき)なのは、会計年度1年を通して欠損している企業かどうか consecutive nan

---

levelを会計年度にするためにmultiindexしなおす

In [25]:
# change index to 会計年度, 四半期
df_2008["会計年度"] = df_2008["決算期"].apply(lambda x: int(x[:4])-1 if int(x[-2:]) <= 3 else int(x[:4])) # 1, 2, 3月なら西暦-1
df_2008["四半期"] = df_2008["決算期"].apply(lambda x: 
                            "Q1" if int(x[-2:]) ==  6 else (
                            "Q2" if int(x[-2:]) ==  9 else (
                            "Q3" if int(x[-2:]) == 12 else (
                            "Q4" if int(x[-2:]) ==  3 else (
                            "NonMar" + x[-2:]
                            ))))
                           )

<ipython-input-25-6295ca30ea3d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2008["会計年度"] = df_2008["決算期"].apply(lambda x: int(x[:4])-1 if int(x[-2:]) <= 3 else int(x[:4])) # 1, 2, 3月なら西暦-1
<ipython-input-25-6295ca30ea3d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2008["四半期"] = df_2008["決算期"].apply(lambda x:


In [26]:
# set multiindex ["企業名", "会計年度", "四半期"]
df_2008.reset_index(inplace=True)
df_2008.set_index(["企業名", "会計年度", "四半期"], inplace=True)

---

In [27]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=[0, 1])[
    df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=[0, 1]) == 1
]

企業名          会計年度
エス・サイエンス     2017   1.00
             2019   1.00
オムロン         2009   1.00
             2012   1.00
             2014   1.00
             2015   1.00
             2019   1.00
コナミホールディングス  2008   1.00
             2010   1.00
             2011   1.00
             2013   1.00
             2014   1.00
             2015   1.00
             2016   1.00
マキタ          2008   1.00
             2009   1.00
             2010   1.00
             2011   1.00
             2012   1.00
             2013   1.00
             2014   1.00
             2015   1.00
             2016   1.00
手間いらず        2014   1.00
             2019   1.00
日本商業開発       2011   1.00
             2012   1.00
Name: 資本的支出, dtype: float64

2008年度以降、1会計年間に１つも資本的支出が観測されていないのは27年

In [28]:
whole_miss_capx = df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=[0, 1])[
    df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=[0, 1]) == 1
].index.get_level_values(0).unique()

whole_miss_capx

Index(['エス・サイエンス', 'オムロン', 'コナミホールディングス', 'マキタ', '手間いらず', '日本商業開発'], dtype='object', name='企業名')

この企業は会計年度1年間丸ごと欠損しているのでサンプルから drop していい

欠損パターンは("Q1", "Q3"), ("Q2", "Q4"), ("Q1", "Q2", "Q3"), ("Q2", "Q3", "Q4"), ("Q1", "Q2", "Q4), ("Q1", "Q3", "Q4)か、1つだけ、もしくは全部(これはdropする)

In [29]:
# previous_fiscal_year = None
# # for i in df_2008.index.get_level_values(0).unique():
# for i in df_2008.index.get_level_values(0).unique():
# #     print(i)
#     for j in df_2008.loc[pd.IndexSlice[i, 2008:, :], :].index.get_level_values(1).unique():
#         temp_df = df_2008.loc[pd.IndexSlice[i, j, :], :]["資本的支出"]
# #         print(temp_df)
#         if temp_df.isna().sum() > 0:
# #             print("missing")
#             missing_type = list(temp_df[temp_df.isna()].index.get_level_values(2).values)
# #             print("missing type: ", temp_df[temp_df.isna()].index.get_level_values(2).values)
#             if missing_type == ["Q1", "Q3"]:
#                 pass
#             elif missing_type == ["Q2", "Q4"]:
#                 pass
#             elif missing_type == ["Q1", "Q2", "Q3"]:
#                 print(i)
#                 print("missing type: ", temp_df[temp_df.isna()].index.get_level_values(2).values)
#                 pass
#             else:
#                 print(i)
#                 print("missing type: ", temp_df[temp_df.isna()].index.get_level_values(2).values)
#                 pass
#         else:
# #             print("no missing")
#             pass

In [30]:
df_2008.loc[pd.IndexSlice["オムロン", :, :], :]["資本的支出"].iloc[0:100]

企業名   会計年度  四半期
オムロン  1999  Q2                  nan
            Q4    31,146,000,000.00
      2000  Q2                  nan
            Q4    37,583,000,000.00
      2001  Q2                  nan
            Q4    38,896,000,000.00
      2002  Q2                  nan
            Q4    34,454,000,000.00
      2003  Q1                  nan
            Q2                  nan
            Q3                  nan
            Q4    38,115,000,000.00
      2004  Q1                  nan
            Q2                  nan
            Q3                  nan
            Q4    38,579,000,000.00
      2005  Q1                  nan
            Q2                  nan
            Q3                  nan
            Q4    40,560,000,000.00
      2006  Q1                  nan
            Q2                  nan
            Q3                  nan
            Q4    44,689,000,000.00
      2007  Q1                  nan
            Q2                  nan
            Q3                  nan
            

In [31]:
df_2008.loc[pd.IndexSlice["システムソフト", :, :], :]["資本的支出"].iloc[0:100]

企業名      会計年度  四半期
システムソフト  1999  Q2               nan
               Q4    143,000,000.00
         2000  Q2      7,000,000.00
               Q4     20,000,000.00
         2001  Q2     26,000,000.00
               Q4     34,000,000.00
         2002  Q2     65,000,000.00
               Q4    101,000,000.00
         2003  Q1      2,000,000.00
               Q2      6,000,000.00
               Q3      5,000,000.00
               Q4      7,000,000.00
         2004  Q1      1,000,000.00
               Q2      5,000,000.00
               Q3      8,000,000.00
               Q4     12,000,000.00
         2005  Q1      8,000,000.00
               Q2      9,000,000.00
               Q3     10,000,000.00
               Q4               nan
         2006  Q1      9,000,000.00
               Q2     26,000,000.00
               Q3     24,000,000.00
               Q4     31,000,000.00
         2007  Q1     36,000,000.00
               Q2     44,000,000.00
               Q3      1,000,000.00
         

In [32]:
df_2008.loc[pd.IndexSlice["ＴＨＫ", :, :], :]["資本的支出"].iloc[0:100]

企業名  会計年度  四半期
ＴＨＫ  1999  Q2                  nan
           Q4     4,857,000,000.00
     2000  Q2     4,009,000,000.00
           Q4    11,076,000,000.00
     2001  Q2     7,647,000,000.00
           Q4     9,226,000,000.00
     2002  Q2     2,578,000,000.00
           Q4     4,759,000,000.00
     2003  Q1     1,624,000,000.00
           Q2     2,500,000,000.00
           Q3     3,802,000,000.00
           Q4     5,105,000,000.00
     2004  Q1     1,774,000,000.00
           Q2     2,663,000,000.00
           Q3     6,001,000,000.00
           Q4     6,963,000,000.00
     2005  Q1     2,712,000,000.00
           Q2     6,989,000,000.00
           Q3     8,524,000,000.00
           Q4    12,520,000,000.00
     2006  Q1     2,601,000,000.00
           Q2     5,386,000,000.00
           Q3     8,845,000,000.00
           Q4    12,848,000,000.00
     2007  Q1    15,305,000,000.00
           Q2     8,995,000,000.00
           Q3    14,327,000,000.00
           Q4    19,618,000,000.00
     

Q4は本決算だから累計かも? --> Q4 < Q2 とかあるので大丈夫なはず

やっぱり四半期報告である以上、報告していない分は時期に含まれている気がするから、年度またいだとしても(つまりQ4の欠損値に対して)次年度を分割して穴埋めする方針で。

In [33]:
# 丸ごと1年欠損している企業をdrop
df_2008 = df_2008.loc[pd.IndexSlice[(recorded_firm_from_2008 - set(whole_miss_capx)), :, :], :]

In [34]:
df_2008.index.get_level_values(0).unique()

Index(['あい　ホールディングス', 'あみやき亭', 'あらた', 'いすゞ自動車', 'いであ', 'いなげや', 'かどや製油',
       'きちりホールディングス', 'きんでん', 'ぐるなび',
       ...
       'ＴＹＫ', 'ＵＡＣＪ', 'ＵＴグループ', 'ＶＴホールディングス', 'ＷＤＢホールディングス', 'ＷＯＷ　ＷＯＲＬＤ',
       'ＷＯＷＯＷ', 'Ｚホールディングス', 'ＺＯＺＯ', 'ｅＢＡＳＥ'],
      dtype='object', name='企業名', length=1430)

ちょうど5の倍数、VRPで割り切れる  
"システムソフト"の欠損が一番多くてバリエーションが多い?っぽいからこいつをうまくできればおっけー

In [35]:
df_2008_copy = df_2008.copy()

In [36]:
df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], :]["資本的支出"]

企業名      会計年度  四半期
システムソフト  1999  Q2               nan
               Q4    143,000,000.00
         2000  Q2      7,000,000.00
               Q4     20,000,000.00
         2001  Q2     26,000,000.00
               Q4     34,000,000.00
         2002  Q2     65,000,000.00
               Q4    101,000,000.00
         2003  Q1      2,000,000.00
               Q2      6,000,000.00
               Q3      5,000,000.00
               Q4      7,000,000.00
         2004  Q1      1,000,000.00
               Q2      5,000,000.00
               Q3      8,000,000.00
               Q4     12,000,000.00
         2005  Q1      8,000,000.00
               Q2      9,000,000.00
               Q3     10,000,000.00
               Q4               nan
         2006  Q1      9,000,000.00
               Q2     26,000,000.00
               Q3     24,000,000.00
               Q4     31,000,000.00
         2007  Q1     36,000,000.00
               Q2     44,000,000.00
               Q3      1,000,000.00
         

In [37]:
# 1企業例
isna_series = df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], "資本的支出"].isna()

for i in range(len(isna_series)-1):
    if (isna_series.iloc[i] == True) & (isna_series.iloc[i+1] == False): 
        # i が nan, i+1 が nan じゃないとき、
        ind_non_nan = df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], :].index[i+1]
        print("index of record be divided: ", ind_non_nan)
        print(df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], "資本的支出"][i], df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], "資本的支出"][i+1])
        num_nan = 0
        ind_nan_list = []
        while isna_series.iloc[i - num_nan]:
            # nan がどのくらい続いていたのかカウントして、
            num_nan += 1
            ind_nan_list.append(df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], "資本的支出"].index[i+1 - num_nan])
        print("num nan: ", num_nan)
        print("index of nan: ", ind_nan_list)
        # nan じゃない i+1 を num_nan + 1の分だけ割っておいて、
        divided_value = (df_2008_copy.loc[pd.IndexSlice[ind_non_nan[0], ind_non_nan[1], ind_non_nan[2]], "資本的支出"] / (num_nan+1))
        print("divided value: ", divided_value)
        df_2008_copy.loc[pd.IndexSlice[ind_non_nan[0], ind_non_nan[1], ind_non_nan[2]], "資本的支出"] = divided_value
        for ind_nan in ind_nan_list:
            # nan にも同じ割った値を代入
            df_2008_copy.loc[pd.IndexSlice[ind_nan[0], ind_nan[1], ind_nan[2]], "資本的支出"] = divided_value

index of record be divided:  ('システムソフト', 1999, 'Q4')
nan 143000000.0
num nan:  1
index of nan:  [('システムソフト', 1999, 'Q2')]
divided value:  71500000.0
index of record be divided:  ('システムソフト', 2006, 'Q1')
nan 9000000.0
num nan:  1
index of nan:  [('システムソフト', 2005, 'Q4')]
divided value:  4500000.0
index of record be divided:  ('システムソフト', 2009, 'Q3')
nan 12000000.0
num nan:  4
index of nan:  [('システムソフト', 2009, 'Q2'), ('システムソフト', 2009, 'Q1'), ('システムソフト', 2008, 'Q4'), ('システムソフト', 2008, 'Q3')]
divided value:  2400000.0
index of record be divided:  ('システムソフト', 2011, 'Q2')
nan 944000000.0
num nan:  3
index of nan:  [('システムソフト', 2011, 'Q1'), ('システムソフト', 2010, 'Q4'), ('システムソフト', 2010, 'Q3')]
divided value:  236000000.0
index of record be divided:  ('システムソフト', 2011, 'Q4')
nan 1000000.0
num nan:  1
index of nan:  [('システムソフト', 2011, 'Q3')]
divided value:  500000.0
index of record be divided:  ('システムソフト', 2012, 'Q2')
nan 21000000.0
num nan:  1
index of nan:  [('システムソフト', 2012, 'Q1')]
divided value:  1

In [38]:
df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], :]["資本的支出"]

企業名      会計年度  四半期
システムソフト  1999  Q2     71,500,000.00
               Q4     71,500,000.00
         2000  Q2      7,000,000.00
               Q4     20,000,000.00
         2001  Q2     26,000,000.00
               Q4     34,000,000.00
         2002  Q2     65,000,000.00
               Q4    101,000,000.00
         2003  Q1      2,000,000.00
               Q2      6,000,000.00
               Q3      5,000,000.00
               Q4      7,000,000.00
         2004  Q1      1,000,000.00
               Q2      5,000,000.00
               Q3      8,000,000.00
               Q4     12,000,000.00
         2005  Q1      8,000,000.00
               Q2      9,000,000.00
               Q3     10,000,000.00
               Q4      4,500,000.00
         2006  Q1      4,500,000.00
               Q2     26,000,000.00
               Q3     24,000,000.00
               Q4     31,000,000.00
         2007  Q1     36,000,000.00
               Q2     44,000,000.00
               Q3      1,000,000.00
         

できた!これを全社でループする

In [39]:
# fill nan (devide after equal) for "資本的支出"

for firm in df_2008.index.get_level_values(0).unique():
    isna_series = df_2008.loc[pd.IndexSlice[firm, :, :], "資本的支出"].isna()

    for i in range(len(isna_series)-1):
        if (isna_series.iloc[i] == True) & (isna_series.iloc[i+1] == False): 
            # i が nan, i+1 が nan じゃないとき、
            ind_non_nan = df_2008.loc[pd.IndexSlice[firm, :, :], :].index[i+1]

            num_nan = 0
            ind_nan_list = []
            while isna_series.iloc[i - num_nan]:
                # nan がどのくらい続いていたのかカウントして、
                num_nan += 1
                ind_nan_list.append(df_2008.loc[pd.IndexSlice[firm, :, :], "資本的支出"].index[i+1 - num_nan])
            # nan じゃない i+1 を num_nan + 1の分だけ割っておいて、
            divided_value = (df_2008.loc[pd.IndexSlice[ind_non_nan[0], ind_non_nan[1], ind_non_nan[2]], "資本的支出"] / (num_nan+1))
            df_2008.loc[pd.IndexSlice[ind_non_nan[0], ind_non_nan[1], ind_non_nan[2]], "資本的支出"] = divided_value

            for ind_nan in ind_nan_list:
                # nan にも同じ割った値を代入
                df_2008.loc[pd.IndexSlice[ind_nan[0], ind_nan[1], ind_nan[2]], "資本的支出"] = divided_value
#     print(firm, " done")

In [40]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], "資本的支出"].isna().sum()

13

In [41]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], "資本的支出"][df_2008.loc[pd.IndexSlice[:, 2008:, :], "資本的支出"].isna()]

企業名              会計年度  四半期
あい　ホールディングス      2020  Q4    nan
インテージホールディングス    2020  Q4    nan
オルバヘルスケアホールディング  2020  Q4    nan
ディア・ライフ          2020  Q3    nan
                       Q4    nan
トーア紡コーポレーション     2020  Q4    nan
パイロットコーポレーション    2020  Q4    nan
フルキャストホールディングス   2020  Q4    nan
メック              2020  Q4    nan
日本エスコン           2020  Q4    nan
鈴木               2020  Q4    nan
ＣＩＪ              2020  Q4    nan
ＩＮＰＥＸ            2020  Q4    nan
Name: 資本的支出, dtype: float64

最後の期2020Q4が欠損していたらどうしようもないが、ぶっちゃけテスト期間になるので欠損したままで良い。


とりあえずここまでで一回保存

In [42]:
df_2008.to_csv("./../data/processed/tse1_2008_capx_labor_filled.csv")

In [3]:
df_2008 = pd.read_csv("./../data/processed/tse1_2008_capx_labor_filled.csv", index_col=[0, 1, 2])

* 期中平均株式数［３ヵ月］

In [10]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], [
    '期中平均株式数［累計］', 
    '期中平均株式数［３ヵ月］', 
    '期末発行済株式総数', 
    '自己株式数'
]].isna().mean()

期中平均株式数［累計］    0.01
期中平均株式数［３ヵ月］   0.61
期末発行済株式総数      0.00
自己株式数          0.03
dtype: float64

In [16]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], '期中平均株式数［累計］'][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], '期中平均株式数［累計］'].isna()
]

企業名          会計年度  四半期
あみやき亭        2013  Q2    nan
あらた          2015  Q1    nan
いすゞ自動車       2014  Q2    nan
きちりホールディングス  2012  Q1    nan
             2013  Q1    nan
                          ..
ＺＯＺＯ         2016  Q2    nan
ｅＢＡＳＥ        2012  Q4    nan
             2017  Q2    nan
             2018  Q4    nan
             2019  Q4    nan
Name: 期中平均株式数［累計］, Length: 885, dtype: float64

In [11]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], '期中平均株式数［３ヵ月］'][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], '期中平均株式数［３ヵ月］'].isna()
]

企業名          会計年度  四半期
あい　ホールディングス  2008  Q1    nan
             2009  Q1    nan
             2010  Q1    nan
             2011  Q1    nan
                   Q3    nan
                          ..
ｅＢＡＳＥ        2019  Q3    nan
                   Q4    nan
             2020  Q2    nan
                   Q3    nan
                   Q4    nan
Name: 期中平均株式数［３ヵ月］, Length: 45331, dtype: float64

In [18]:
# 累計も3ヶ月も欠損しているレコード
df_2008.loc[pd.IndexSlice[:, 2008:, :], ['期中平均株式数［累計］', '期中平均株式数［３ヵ月］']][
    (df_2008.loc[pd.IndexSlice[:, 2008:, :], '期中平均株式数［累計］'].isna()) & (df_2008.loc[pd.IndexSlice[:, 2008:, :], '期中平均株式数［３ヵ月］'].isna())
]

期中平均株式数［累計］  期中平均株式数［３ヵ月］
企業名         会計年度 四半期                           
あみやき亭       2013 Q2           nan           nan
あらた         2015 Q1           nan           nan
いすゞ自動車      2014 Q2           nan           nan
きちりホールディングス 2012 Q1           nan           nan
            2013 Q1           nan           nan
...                           ...           ...
ＺＯＺＯ        2016 Q2           nan           nan
ｅＢＡＳＥ       2012 Q4           nan           nan
            2017 Q2           nan           nan
            2018 Q4           nan           nan
            2019 Q4           nan           nan

[885 rows x 2 columns]

'期中平均株式数［累計］'の欠損レコード(885個)は'期中平均株式数［３ヵ月］'も必ず欠損している。

In [19]:
c = collections.Counter(list(df_2008.loc[pd.IndexSlice[:, 2008:, :], '期中平均株式数［累計］'][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], '期中平均株式数［累計］'].isna()
].index.get_level_values(0)))

c.values()

dict_values([1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 2, 1, 5, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 6, 3, 1, 1, 1, 3, 2, 2, 2, 2, 2, 1, 7, 3, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 4, 2, 1, 1, 1, 1, 2, 1, 1, 1, 4, 1, 2, 4, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 2, 3, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 3, 3, 1, 1, 1, 1, 5, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 4, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

累計は欠損少ない

In [20]:
c.most_common()[:50]

[('ＣＯＴＡ', 9),
 ('エムスリー', 7),
 ('インフォマート', 6),
 ('ＫＤＤＩ', 6),
 ('ＬＩＦＵＬＬ', 6),
 ('アステラス製薬', 5),
 ('フジオフードグループ本社', 5),
 ('参天製薬', 5),
 ('ガンホー・オンライン・エンター', 4),
 ('クレハ', 4),
 ('グリムス', 4),
 ('プレステージ・インターナショナ', 4),
 ('メンバーズ', 4),
 ('日本たばこ産業', 4),
 ('日本ハム', 4),
 ('日本Ｍ＆Ａセンター', 4),
 ('日進工具', 4),
 ('朝日インテック', 4),
 ('ＧＭＯペイメントゲートウェイ', 4),
 ('ＭｏｎｏｔａＲＯ', 4),
 ('ｅＢＡＳＥ', 4),
 ('きちりホールディングス', 3),
 ('ウェルネット', 3),
 ('エス・エム・エス', 3),
 ('エムティーアイ', 3),
 ('サイバーエージェント', 3),
 ('ジェイエイシーリクルートメント', 3),
 ('ティア', 3),
 ('テクマトリックス', 3),
 ('ディア・ライフ', 3),
 ('ディーブイエックス', 3),
 ('ピーシーデポコーポレーション', 3),
 ('ファンコミュニケーションズ', 3),
 ('リンクアンドモチベーション', 3),
 ('レーザーテック', 3),
 ('伊藤忠テクノソリューションズ', 3),
 ('住友ベークライト', 3),
 ('夢真ビーネックスグループ', 3),
 ('山洋電気', 3),
 ('日本ライフライン', 3),
 ('穴吹興産', 3),
 ('野村総合研究所', 3),
 ('ＪＣＵ', 3),
 ('アイティメディア', 2),
 ('アウトソーシング', 2),
 ('アドソル日進', 2),
 ('アドバンテッジリスクマネジメン', 2),
 ('アバント', 2),
 ('アルプス技研', 2),
 ('アークランドサービスホールディ', 2)]

In [24]:
df_2008.loc[pd.IndexSlice[c.most_common()[1][0], 2008:, :], [
    '期中平均株式数［累計］', 
    '期中平均株式数［３ヵ月］', 
    '期末発行済株式総数', 
    '自己株式数'
]]

期中平均株式数［累計］   期中平均株式数［３ヵ月］      期末発行済株式総数     自己株式数
企業名   会計年度 四半期                                                       
エムスリー 2008 Q1      260,928.00     260,928.00     260,928.00       nan
           Q2      261,027.00     261,125.00     261,270.00       nan
           Q3      261,108.00     261,270.00     261,270.00       nan
           Q4      261,176.00            nan     261,666.00       nan
      2009 Q1      261,666.00     261,666.00     261,666.00       nan
           Q2      261,678.00     261,690.00     261,690.00       nan
           Q3      261,683.00     261,694.00     261,696.00       nan
           Q4      261,689.00            nan     261,732.00       nan
      2010 Q1      261,853.00     261,853.00     261,984.00       nan
           Q2      261,918.00     261,984.00     261,984.00       nan
           Q3      261,944.00     261,984.00     262,020.00       nan
           Q4      261,963.00            nan     262,020.00       nan
      2011 Q1      262,056.00     262,056.00     262,086.00       nan
           Q2             nan            nan     264,201.00       nan
           Q3      526,054.00            nan     528,604.00       nan
           Q4      526,705.00            nan     528,628.00       nan
      2012 Q1      528,679.00     528,679.00     528,750.00       nan
           Q2             nan            nan     529,508.00       nan
           Q3    1,588,079.00            nan   1,588,686.00       nan
           Q4    1,587,541.00            nan   1,588,926.00       nan
      2013 Q1             nan            nan   1,589,532.00       nan
           Q2             nan            nan   1,589,958.00       nan
           Q3             nan            nan   1,592,136.00       nan
           Q4             nan            nan   1,616,315.00    162.00
      2014 Q1  323,256,077.00 323,256,077.00 323,313,400.00 32,400.00
           Q2  323,281,495.00 323,306,636.00 323,375,200.00 32,400.00
           Q3  323,304,341.00 323,349,793.00 323,393,800.00 32,400.00
           Q4  323,328,834.00            nan 323,499,400.00 32,400.00
      2015 Q1  323,523,438.00 323,523,438.00 323,597,800.00 32,400.00
           Q2  323,544,929.00 323,566,187.00 323,600,600.00 32,400.00
           Q3  323,561,945.00 323,595,787.00 323,636,600.00 32,400.00
           Q4  323,574,075.00            nan 323,646,000.00 32,400.00
      2016 Q1  323,640,795.00 323,640,795.00 323,708,600.00 32,400.00
           Q2  323,668,703.00 323,696,312.00 323,748,900.00 32,400.00
           Q3  323,685,520.00 323,718,974.00 323,761,300.00 32,400.00
           Q4  323,701,536.00            nan 323,790,100.00 32,400.00
      2017 Q1  323,773,547.00 323,773,547.00 323,811,900.00 32,400.00
           Q2  323,777,422.00 323,781,260.00 323,814,900.00 32,400.00
           Q3  323,790,148.00 323,815,452.00 323,883,800.00 32,400.00
           Q4  323,807,332.00            nan 323,897,900.00 32,400.00
      2018 Q1  323,871,123.00 323,871,123.00 323,915,100.00 32,400.00
           Q2             nan            nan 323,917,400.00       nan
           Q3  647,788,600.00 647,840,689.00 647,951,800.00 65,002.00
           Q4  647,810,235.00            nan 647,957,200.00 65,002.00
      2019 Q1  678,396,682.00 678,396,682.00 678,463,200.00 65,002.00
           Q2  678,406,283.00 678,415,848.00 678,490,400.00 65,002.00
           Q3  678,421,743.00 678,452,485.00 678,564,200.00 65,002.00
           Q4  678,454,248.00            nan 678,645,600.00 65,002.00
      2020 Q1  678,599,342.00 678,599,342.00 678,686,400.00 65,068.00
           Q2  678,624,694.00 678,649,767.00 678,717,400.00 44,799.00
           Q3  678,644,883.00 678,685,043.00 678,747,600.00 44,867.00
           Q4  678,664,147.00            nan 678,785,600.00 45,067.00

累計の欠損をmeanやffillで埋める(limit=1)なら、この企業はdropする。もしくは期末を使うか。<-- 期末のほうがいいかも

In [28]:
# 累計も'期末発行済株式総数も欠損しているレコード
df_2008.loc[pd.IndexSlice[:, 2008:, :], ['期中平均株式数［累計］', '期末発行済株式総数', '自己株式数']][
    (df_2008.loc[pd.IndexSlice[:, 2008:, :], '期中平均株式数［累計］'].isna()) & (df_2008.loc[pd.IndexSlice[:, 2008:, :], '期末発行済株式総数'].isna())
]

,,,期中平均株式数［累計］,期末発行済株式総数,自己株式数
企業名,会計年度,四半期,,,
イトーキ,2008,Q2,nan,nan,nan
グリムス,2008,Q1,nan,nan,nan
システムソフト,2008,Q1,nan,nan,nan
ビーピー・カストロール,2008,Q2,nan,nan,nan
フジオフードグループ本社,2008,Q2,nan,nan,nan
フューチャー,2008,Q2,nan,nan,nan
ラックランド,2008,Q2,nan,nan,nan
大末建設,2014,Q1,nan,nan,nan


累計も期末も欠損しているのはmeanかffillする(期末はffillがいいかもね)

In [12]:
c = collections.Counter(list(df_2008.loc[pd.IndexSlice[:, 2008:, :], '期中平均株式数［３ヵ月］'][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], '期中平均株式数［３ヵ月］'].isna()
].index.get_level_values(0)))

c.values()

dict_values([33, 33, 34, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 21, 33, 33, 33, 23, 33, 33, 33, 33, 33, 33, 33, 25, 33, 33, 33, 33, 33, 23, 32, 33, 20, 23, 33, 33, 33, 33, 33, 33, 13, 33, 33, 33, 35, 25, 33, 33, 33, 33, 34, 13, 33, 33, 33, 33, 33, 33, 33, 33, 33, 13, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 34, 33, 6, 33, 33, 33, 33, 33, 33, 33, 33, 33, 27, 21, 33, 33, 25, 33, 33, 33, 33, 33, 33, 34, 33, 33, 26, 33, 33, 33, 33, 33, 19, 34, 21, 33, 33, 33, 33, 33, 33, 33, 17, 33, 33, 33, 33, 33, 33, 34, 33, 33, 23, 33, 33, 33, 33, 33, 33, 33, 33, 33, 14, 26, 33, 33, 31, 33, 33, 33, 33, 33, 34, 33, 33, 32, 33, 33, 33, 33, 15, 33, 33, 33, 33, 25, 33, 33, 33, 33, 33, 33, 33, 13, 33, 33, 32, 33, 33, 33, 24, 33, 33, 34, 33, 35, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 27, 33, 33, 33, 13, 33, 33, 13, 33, 33, 33, 33, 21, 33, 33, 33, 33, 33, 33, 32, 33, 32, 33, 33, 32, 33, 33, 25, 33, 33, 33, 33, 33, 33, 33, 33, 34, 33, 33, 33, 33, 33, 33, 33, 

3ヶ月は各社30期ぐらい落ちてる

In [13]:
c.most_common()[:50]

[('ファルコホールディングス', 36),
 ('アバント', 35),
 ('グリムス', 35),
 ('ファンコミュニケーションズ', 35),
 ('三菱食品', 35),
 ('伊藤忠食品', 35),
 ('ＭｏｎｏｔａＲＯ', 35),
 ('あらた', 34),
 ('アルコニックス', 34),
 ('イフジ産業', 34),
 ('エス・エム・エス', 34),
 ('エフ・ジェー・ネクスト', 34),
 ('オプテックスグループ', 34),
 ('ガンホー・オンライン・エンター', 34),
 ('クワザワホールディングス', 34),
 ('サンネクスタグループ', 34),
 ('シークス', 34),
 ('シーティーエス', 34),
 ('シード', 34),
 ('ソフトクリエイトホールディング', 34),
 ('タカラレーベン', 34),
 ('タケエイ', 34),
 ('チノー', 34),
 ('ティア', 34),
 ('テクマトリックス', 34),
 ('ディア・ライフ', 34),
 ('ノジマ', 34),
 ('ビジネスエンジニアリング', 34),
 ('ビジネスブレイン太田昭和', 34),
 ('フクシマガリレイ', 34),
 ('フジオフードグループ本社', 34),
 ('ブルドックソース', 34),
 ('プロトコーポレーション', 34),
 ('ミクシィ', 34),
 ('ミスミグループ本社', 34),
 ('モーニングスター', 34),
 ('ユニカフェ', 34),
 ('大末建設', 34),
 ('東急', 34),
 ('東海理化電機製作所', 34),
 ('藤久', 34),
 ('西日本旅客鉄道', 34),
 ('豊和工業', 34),
 ('野村総合研究所', 34),
 ('ＤＴＳ', 34),
 ('ＧＭＯグローバルサイン・ホール', 34),
 ('ＧＭＯペパボ', 34),
 ('ＪＰホールディングス', 34),
 ('ＫＮＴ－ＣＴホールディングス', 34),
 ('ＵＴグループ', 34)]

In [26]:
df_2008.loc[pd.IndexSlice[c.most_common()[1][0], 2008:, :], [
    '期中平均株式数［累計］', 
    '期中平均株式数［３ヵ月］', 
    '期末発行済株式総数', 
    '自己株式数'
]]

期中平均株式数［累計］   期中平均株式数［３ヵ月］      期末発行済株式総数     自己株式数
企業名   会計年度 四半期                                                       
エムスリー 2008 Q1      260,928.00     260,928.00     260,928.00       nan
           Q2      261,027.00     261,125.00     261,270.00       nan
           Q3      261,108.00     261,270.00     261,270.00       nan
           Q4      261,176.00            nan     261,666.00       nan
      2009 Q1      261,666.00     261,666.00     261,666.00       nan
           Q2      261,678.00     261,690.00     261,690.00       nan
           Q3      261,683.00     261,694.00     261,696.00       nan
           Q4      261,689.00            nan     261,732.00       nan
      2010 Q1      261,853.00     261,853.00     261,984.00       nan
           Q2      261,918.00     261,984.00     261,984.00       nan
           Q3      261,944.00     261,984.00     262,020.00       nan
           Q4      261,963.00            nan     262,020.00       nan
      2011 Q1      262,056.00     262,056.00     262,086.00       nan
           Q2             nan            nan     264,201.00       nan
           Q3      526,054.00            nan     528,604.00       nan
           Q4      526,705.00            nan     528,628.00       nan
      2012 Q1      528,679.00     528,679.00     528,750.00       nan
           Q2             nan            nan     529,508.00       nan
           Q3    1,588,079.00            nan   1,588,686.00       nan
           Q4    1,587,541.00            nan   1,588,926.00       nan
      2013 Q1             nan            nan   1,589,532.00       nan
           Q2             nan            nan   1,589,958.00       nan
           Q3             nan            nan   1,592,136.00       nan
           Q4             nan            nan   1,616,315.00    162.00
      2014 Q1  323,256,077.00 323,256,077.00 323,313,400.00 32,400.00
           Q2  323,281,495.00 323,306,636.00 323,375,200.00 32,400.00
           Q3  323,304,341.00 323,349,793.00 323,393,800.00 32,400.00
           Q4  323,328,834.00            nan 323,499,400.00 32,400.00
      2015 Q1  323,523,438.00 323,523,438.00 323,597,800.00 32,400.00
           Q2  323,544,929.00 323,566,187.00 323,600,600.00 32,400.00
           Q3  323,561,945.00 323,595,787.00 323,636,600.00 32,400.00
           Q4  323,574,075.00            nan 323,646,000.00 32,400.00
      2016 Q1  323,640,795.00 323,640,795.00 323,708,600.00 32,400.00
           Q2  323,668,703.00 323,696,312.00 323,748,900.00 32,400.00
           Q3  323,685,520.00 323,718,974.00 323,761,300.00 32,400.00
           Q4  323,701,536.00            nan 323,790,100.00 32,400.00
      2017 Q1  323,773,547.00 323,773,547.00 323,811,900.00 32,400.00
           Q2  323,777,422.00 323,781,260.00 323,814,900.00 32,400.00
           Q3  323,790,148.00 323,815,452.00 323,883,800.00 32,400.00
           Q4  323,807,332.00            nan 323,897,900.00 32,400.00
      2018 Q1  323,871,123.00 323,871,123.00 323,915,100.00 32,400.00
           Q2             nan            nan 323,917,400.00       nan
           Q3  647,788,600.00 647,840,689.00 647,951,800.00 65,002.00
           Q4  647,810,235.00            nan 647,957,200.00 65,002.00
      2019 Q1  678,396,682.00 678,396,682.00 678,463,200.00 65,002.00
           Q2  678,406,283.00 678,415,848.00 678,490,400.00 65,002.00
           Q3  678,421,743.00 678,452,485.00 678,564,200.00 65,002.00
           Q4  678,454,248.00            nan 678,645,600.00 65,002.00
      2020 Q1  678,599,342.00 678,599,342.00 678,686,400.00 65,068.00
           Q2  678,624,694.00 678,649,767.00 678,717,400.00 44,799.00
           Q3  678,644,883.00 678,685,043.00 678,747,600.00 44,867.00
           Q4  678,664,147.00            nan 678,785,600.00 45,067.00

累計から3ヶ月の求め方

$$\begin{eqnarray}
    NI_{Q} &:& Q四半期3ヶ月の当期純利益 \\
    N_{M} &:& M月末株式数 \ (発行済-自己株) \\
    MN_{Q} &:& Q四半期3ヶ月の期中平均株式数 \\
    CMN_{Q} &:& Q四半期累計の期中平均株式数 \\
    \\
    MN_{Q1}をCMNから求めるときとき \\
    MN_{Q1} &=& CMN_{Q1} \tag{1} \\
    \\
    MN_{Q2}をCMNから求めるときとき \\
    CMN_{Q2} - CMN_{Q1} &=& \frac{N_{4}+N_{5}+N_{6}+N_{7}+N_{8}+N_{9}}{6} - \frac{N_{4}+N_{5}+N_{6}}{3} \\
        &=& \frac{N_{4}+N_{5}+N_{6}+N_{7}+N_{8}+N_{9}}{6} - \frac{2N_{4}+2N_{5}+2N_{6}}{6} \\
        &=& \frac{1}{6} (N_{7}+N_{8}+N_{9}) - \frac{1}{6} (N_{4}+N_{5}+N_{6}) \\
        &=& \frac{1}{2} \frac{(N_{7}+N_{8}+N_{9})}{3} - \frac{1}{2} \frac{(N_{4}+N_{5}+N_{6})}{3} \\
        &=& \frac{1}{2} MN_{Q2} - \frac{1}{2} CMN_{Q1} \\
        (&=& \frac{1}{2} MN_{Q2} - \frac{1}{2} MN_{Q1}) \\
    よって
    CMN_{Q2} - CMN_{Q1} &=& \frac{1}{2} MN_{Q2} - \frac{1}{2} CMN_{Q1} \\
    2CMN_{Q2} - 2CMN_{Q1} &=& MN_{Q2} - CMN_{Q1} \\
    MN_{Q2} &=& 2CMN_{Q2} - CMN_{Q1} \tag{2} \\
    \\
    MN_{Q3}をCMNから求めるときとき \\
    CMN_{Q3} - CMN_{Q2} &=& \frac{N_{4}+N_{5}+N_{6}+N_{7}+N_{8}+N_{9}+N_{10}+N_{11}+N_{12}}{9} - \frac{N_{4}+N_{5}+N_{6}+N_{7}+N_{8}+N_{9}}{6} \\
        &=& \frac{2N_{4}+2N_{5}+2N_{6}+2N_{7}+2N_{8}+2N_{9}+2N_{10}+2N_{11}+2N_{12}}{18} - \frac{3N_{4}+3N_{5}+3N_{6}+3N_{7}+3N_{8}+3N_{9}}{18} \\
        &=& \frac{1}{9} (N_{10}+N_{11}+N_{12}) - \frac{1}{18} (N_{4}+N_{5}+N_{6}+N_{7}+N_{8}+N_{9}) \\
        &=& \frac{1}{3} \frac{(N_{10}+N_{11}+N_{12})}{3} - \frac{1}{3} \frac{(N_{4}+N_{5}+N_{6}+N_{7}+N_{8}+N_{9})}{6} \\
        &=& \frac{1}{3} MN_{Q3} - \frac{1}{3} CMN_{Q2} \\
    よって
    CMN_{Q3} - CMN_{Q2} &=& \frac{1}{3} MN_{Q3} - \frac{1}{3} CMN_{Q2} \\
    3CMN_{Q3} - 3CMN_{Q2} &=& MN_{Q3} - CMN_{Q2} \\
    MN_{Q3} &=& 3CMN_{Q3} - 2CMN_{Q2} \tag{3} \\
    \\
    MN_{Q4}をCMNから求めるときとき \\
    CMN_{Q4} - CMN_{Q3} &=& \frac{N_{4}+N_{5}+N_{6}+N_{7}+N_{8}+N_{9}+N_{10}+N_{11}+N_{12}+N_{1}+N_{2}+N_{3}}{12} - \frac{N_{4}+N_{5}+N_{6}+N_{7}+N_{8}+N_{9}+N_{10}+N_{11}+N_{12}}{9} \\
        &=& \frac{3N_{4}+3N_{5}+3N_{6}+3N_{7}+3N_{8}+3N_{9}+3N_{10}+3N_{11}+3N_{12}+3N_{1}+3N_{2}+3N_{3}}{36} - \frac{4N_{4}+4N_{5}+4N_{6}+4N_{7}+4N_{8}+4N_{9}+4N_{10}+4N_{11}+4N_{12}}{36} \\
        &=& \frac{1}{12} (N_{1}+N_{2}+N_{3}) - \frac{1}{36} (N_{4}+N_{5}+N_{6}+N_{7}+N_{8}+N_{9}+N_{10}+N_{11}+N_{12}) \\
        &=& \frac{1}{4} \frac{(N_{1}+N_{2}+N_{3})}{3} - \frac{1}{4} \frac{(N_{4}+N_{5}+N_{6}+N_{7}+N_{8}+N_{9}+N_{10}+N_{11}+N_{12})}{9} \\
        &=& \frac{1}{4} MN_{Q4} - \frac{1}{4} CMN_{Q3} \\
    よって
    CMN_{Q4} - CMN_{Q3} &=& \frac{1}{4} MN_{Q4} - \frac{1}{4} CMN_{Q3} \\
    4CMN_{Q4} - 4CMN_{Q3} &=& MN_{Q4} - CMN_{Q3} \\
    MN_{Q4} &=& 4CMN_{Q4} - 3CMN_{Q3} \tag{4} \\
\end{eqnarray}$$

* 四半期EPS

In [4]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], ['１株当たり利益［３ヵ月］', 
                                         '１株当たり利益［累計］', 
                                         '当期純利益（連結）［累計］', 
                                         '【ＱＴＲ】当期利益', 
                                         '期中平均株式数［累計］', 
                                         '期中平均株式数［３ヵ月］', 
                                         '期末発行済株式総数', 
                                         '自己株式数'
                                        ]].isna().mean()

１株当たり利益［３ヵ月］    0.03
１株当たり利益［累計］     0.01
当期純利益（連結）［累計］   0.20
【ＱＴＲ】当期利益       0.00
期中平均株式数［累計］     0.01
期中平均株式数［３ヵ月］    0.61
期末発行済株式総数       0.00
自己株式数           0.03
dtype: float64

In [5]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], '１株当たり利益［３ヵ月］'][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], '１株当たり利益［３ヵ月］'].isna()
]

企業名          会計年度  四半期
あい　ホールディングス  2008  Q1    nan
あみやき亭        2009  Q3    nan
             2013  Q2    nan
             2020  Q4    nan
あらた          2014  Q1    nan
                          ..
ｅＢＡＳＥ        2012  Q4    nan
             2017  Q2    nan
             2018  Q4    nan
             2019  Q4    nan
             2020  Q4    nan
Name: １株当たり利益［３ヵ月］, Length: 2293, dtype: float64

In [6]:
c = collections.Counter(list(df_2008.loc[pd.IndexSlice[:, 2008:, :], '１株当たり利益［３ヵ月］'][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], '１株当たり利益［３ヵ月］'].isna()
].index.get_level_values(0)))

c.values()

dict_values([1, 3, 3, 1, 3, 4, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 2, 1, 3, 1, 1, 3, 1, 2, 4, 1, 7, 2, 1, 1, 3, 3, 5, 1, 1, 5, 2, 2, 1, 2, 2, 3, 2, 2, 2, 2, 1, 3, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 1, 5, 1, 1, 1, 5, 3, 1, 1, 1, 2, 1, 3, 2, 3, 2, 2, 1, 9, 4, 2, 2, 2, 3, 1, 2, 2, 1, 2, 1, 3, 3, 1, 3, 3, 1, 3, 1, 3, 2, 1, 7, 5, 2, 1, 2, 6, 2, 1, 1, 2, 3, 1, 1, 2, 4, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 3, 1, 1, 1, 2, 2, 1, 1, 3, 7, 1, 2, 4, 3, 2, 3, 3, 1, 5, 1, 1, 8, 3, 1, 2, 2, 3, 1, 1, 5, 1, 2, 3, 5, 1, 1, 2, 1, 2, 2, 3, 1, 2, 4, 3, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 4, 3, 1, 1, 1, 5, 1, 2, 1, 2, 1, 5, 1, 2, 2, 1, 1, 2, 3, 1, 2, 1, 1, 3, 2, 3, 3, 3, 1, 1, 1, 3, 2, 2, 2, 2, 2, 4, 2, 2, 6, 2, 1, 4, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 4, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 3, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 4, 1, 1, 3, 1, 1, 2, 1, 2, 1, 3, 1, 2, 1, 1, 2, 2, 3, 2, 2, 4, 3, 2, 1, 1, 5, 2, 2, 1, 2, 2, 3, 2, 4, 4, 1, 1, 1, 3, 6, 1, 4, 1, 1, 4, 5, 2, 1, 2, 1, 2, 1, 2, 1, 1, 

各社欠損はあるけど、数は1~5くらいで少ない(10落ちてるやつはある)

In [7]:
c.most_common()[:50]

[('ＣＯＴＡ', 10),
 ('インフォマート', 9),
 ('キーエンス', 8),
 ('アウトソーシング', 7),
 ('エムスリー', 7),
 ('ガンホー・オンライン・エンター', 7),
 ('ファンコミュニケーションズ', 7),
 ('リンクアンドモチベーション', 7),
 ('ＭｏｎｏｔａＲＯ', 7),
 ('エン・ジャパン', 6),
 ('ジェイエイシーリクルートメント', 6),
 ('ディア・ライフ', 6),
 ('フジオフードグループ本社', 6),
 ('ペッパーフードサービス', 6),
 ('住友ゴム工業', 6),
 ('日本たばこ産業', 6),
 ('沖縄電力', 6),
 ('電算', 6),
 ('ＡＧＣ', 6),
 ('ＧＭＯペイメントゲートウェイ', 6),
 ('ＫＤＤＩ', 6),
 ('ＬＩＦＵＬＬ', 6),
 ('アサヒグループホールディングス', 5),
 ('アステラス製薬', 5),
 ('アルプス技研', 5),
 ('アークランドサービスホールディ', 5),
 ('エムティーアイ', 5),
 ('キリンホールディングス', 5),
 ('クレハ', 5),
 ('グリムス', 5),
 ('サイバーエージェント', 5),
 ('サッポロホールディングス', 5),
 ('ティア', 5),
 ('デジタルホールディングス', 5),
 ('トリドールホールディングス', 5),
 ('トーア紡コーポレーション', 5),
 ('ハイパー', 5),
 ('バリューコマース', 5),
 ('パイロットコーポレーション', 5),
 ('ブロンコビリー', 5),
 ('プレステージ・インターナショナ', 5),
 ('ミルボン', 5),
 ('メンバーズ', 5),
 ('住友ベークライト', 5),
 ('内外トランスライン', 5),
 ('参天製薬', 5),
 ('多木化学', 5),
 ('大塚商会', 5),
 ('山洋電気', 5),
 ('日本ライフライン', 5)]

In [8]:
df_2008.loc[pd.IndexSlice[c.most_common()[1][0], 2008:, :], [
    '１株当たり利益［３ヵ月］', 
    '１株当たり利益［累計］', 
    '当期純利益（連結）［累計］', 
    '【ＱＴＲ】当期利益', 
    '期中平均株式数［累計］', 
    '期中平均株式数［３ヵ月］', 
    '期末発行済株式総数', 
    '自己株式数'
]]

１株当たり利益［３ヵ月］  １株当たり利益［累計］    当期純利益（連結）［累計］       【ＱＴＲ】当期利益  \
企業名     会計年度 四半期                                                               
インフォマート 2008 Q1            nan     4,719.16              nan   97,000,000.00   
             Q2            nan     7,363.55              nan   96,000,000.00   
             Q3            nan    10,716.54              nan  120,000,000.00   
             Q4       2,729.09     2,729.09              nan   94,000,000.00   
        2009 Q1       2,806.49     5,535.38              nan             nan   
             Q2       3,021.58     8,556.38              nan  103,000,000.00   
             Q3       3,229.83    11,786.29              nan  112,000,000.00   
             Q4       3,111.56     3,111.56              nan  108,000,000.00   
        2010 Q1       2,398.83     5,510.39              nan   84,000,000.00   
             Q2       2,235.11     7,745.50              nan   78,000,000.00   
             Q3       2,360.18    10,105.39              nan   83,000,000.00   
             Q4       2,068.83     2,068.83    52,000,000.00   72,000,000.00   
        2011 Q1       2,234.54     4,303.37   107,000,000.00   79,000,000.00   
             Q2       2,640.50     6,943.87   178,000,000.00   93,000,000.00   
             Q3       3,288.24    10,234.03   261,000,000.00  116,000,000.00   
             Q4       2,569.76     2,569.76    73,000,000.00   91,000,000.00   
        2012 Q1       2,472.98     5,042.74   137,000,000.00   89,000,000.00   
             Q2       1,306.13     6,348.87   131,000,000.00   46,000,000.00   
             Q3            nan          nan   390,000,000.00  270,000,000.00   
             Q4          21.48        21.48   144,000,000.00  155,000,000.00   
        2013 Q1            nan          nan   309,000,000.00  166,000,000.00   
             Q2          12.22        34.49   479,000,000.00  177,000,000.00   
             Q3            nan          nan   618,000,000.00  133,000,000.00   
             Q4           8.24         8.24   239,000,000.00  243,000,000.00   
        2014 Q1           8.80        17.04   493,000,000.00  260,000,000.00   
             Q2          11.65        28.69   832,000,000.00  344,000,000.00   
             Q3            nan          nan 1,150,000,000.00  330,000,000.00   
             Q4           5.09         5.09   307,000,000.00  307,000,000.00   
        2015 Q1           5.06        10.16   612,000,000.00  305,000,000.00   
             Q2           5.66        15.81   954,000,000.00  342,000,000.00   
             Q3           5.55        21.38 1,308,000,000.00  354,000,000.00   
             Q4           4.55         4.55   294,000,000.00  294,000,000.00   
        2016 Q1           4.71         9.26   600,000,000.00  306,000,000.00   
             Q2           4.47        13.73   890,000,000.00  290,000,000.00   
             Q3            nan          nan 1,205,000,000.00  315,000,000.00   
             Q4           2.63         2.63   341,000,000.00  341,000,000.00   
        2017 Q1          -4.32        -1.62  -207,000,000.00 -548,000,000.00   
             Q2           1.92         0.09    11,000,000.00  218,000,000.00   
             Q3           3.26         3.17   384,000,000.00  373,000,000.00   
             Q4           2.72         2.72   311,000,000.00  311,000,000.00   
        2018 Q1           3.14         5.86   670,000,000.00  359,000,000.00   
             Q2           3.63         9.49 1,084,000,000.00  414,000,000.00   
             Q3           4.09        13.59 1,552,000,000.00  468,000,000.00   
             Q4           5.26         5.26   601,000,000.00  601,000,000.00   
        2019 Q1           4.00         9.26 1,058,000,000.00  457,000,000.00   
             Q2           4.33        13.59 1,553,000,000.00  495,000,000.00   
             Q3            nan          nan 1,694,000,000.00  141,000,000.00   
             Q4           1.46         1.46   333,000,000.00  333,000,000.00   
        2020 Q1   

累計EPSを求める際の期中平均株式数も、累計期間中の平均株式数なので、単純に累計EPSの差分をとって3ヶ月EPSにするのは良くなさそう  

今はとりあえず "【ＱＴＲ】当期利益" / "期中平均株式数［３ヵ月］"で代入できるところは代入  
"期中平均株式数［３ヵ月］"が欠損している場合は 月別の"期末発行済株式総数", "自己株式数"を取得して算出。  
どっちにしろ最後のスケーリング処理にも"期中平均株式数［３ヵ月］"を使うので、結局"期中平均株式数［３ヵ月］"の欠損は完璧に埋めないといけない。

In [9]:
# これを代入
df_2008.loc[pd.IndexSlice[c.most_common()[1], 2008:, :], "【ＱＴＲ】当期利益"] / df_2008.loc[pd.IndexSlice[c.most_common()[1], 2008:, :], "期中平均株式数［３ヵ月］"]

企業名      会計年度  四半期
インフォマート  2008  Q1         nan
               Q2         nan
               Q3         nan
               Q4    2,711.12
         2009  Q1         nan
               Q2    2,995.93
               Q3         nan
               Q4    3,091.46
         2010  Q1    2,404.47
               Q2    2,232.72
               Q3         nan
               Q4    2,043.71
         2011  Q1    2,242.41
               Q2    2,639.80
               Q3         nan
               Q4    2,545.81
         2012  Q1         nan
               Q2         nan
               Q3         nan
               Q4       21.45
         2013  Q1         nan
               Q2         nan
               Q3         nan
               Q4        8.23
         2014  Q1         nan
               Q2         nan
               Q3         nan
               Q4        5.09
         2015  Q1         nan
               Q2         nan
               Q3         nan
               Q4        4.53
         2016  Q1    

* 棚卸資産

In [50]:
# 欠損率
df_2008.loc[pd.IndexSlice[:, 2008:, :], "棚卸資産"].isna().mean()

0.0285771920387305

In [51]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], "棚卸資産"][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], "棚卸資産"].isna()
]

企業名    会計年度  四半期
ぐるなび   2016  Q1    nan
             Q2    nan
             Q3    nan
             Q4    nan
       2017  Q1    nan
                    ..
ｅＢＡＳＥ  2011  Q1    nan
             Q2    nan
       2014  Q4    nan
       2018  Q4    nan
       2020  Q4    nan
Name: 棚卸資産, Length: 2125, dtype: float64

In [52]:
c = collections.Counter(list(df_2008.loc[pd.IndexSlice[:, 2008:, :], "棚卸資産"][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], "棚卸資産"].isna()
].index.get_level_values(0)))

c.values()

dict_values([20, 3, 7, 15, 3, 3, 3, 27, 4, 3, 52, 3, 1, 3, 3, 1, 3, 3, 3, 1, 18, 3, 3, 24, 42, 32, 3, 3, 43, 3, 32, 3, 52, 3, 3, 1, 52, 20, 3, 3, 3, 27, 3, 2, 3, 29, 3, 11, 3, 3, 49, 3, 3, 3, 3, 3, 41, 15, 3, 11, 3, 3, 3, 3, 9, 3, 3, 2, 3, 52, 3, 3, 40, 12, 3, 1, 50, 3, 3, 3, 1, 34, 3, 3, 1, 3, 3, 3, 3, 51, 3, 3, 39, 3, 3, 3, 3, 3, 3, 3, 3, 8, 3, 1, 3, 52, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 37, 1, 3, 3, 3, 3, 3, 52, 3, 50, 3, 3, 3, 52, 3, 3, 3, 3, 3, 36, 3, 3, 3, 3, 3, 3, 3, 52, 3, 3, 3, 3, 3, 3, 3, 19, 3, 3, 1, 39, 3, 52, 3, 3, 46, 52, 3, 3, 1, 3, 3, 3, 52, 2, 2, 3, 1, 2, 47, 52, 22, 29, 3, 3, 3, 3, 1, 3, 28, 3, 3, 3, 1, 3, 3, 3, 11, 3, 2, 1, 10, 11])

In [53]:
c.most_common()[:50]

[('アドバンテッジリスクマネジメン', 52),
 ('クイック', 52),
 ('グリー', 52),
 ('バリューコマース', 52),
 ('京阪神ビルディング', 52),
 ('夢真ビーネックスグループ', 52),
 ('川西倉庫', 52),
 ('日本Ｍ＆Ａセンター', 52),
 ('楽天グループ', 52),
 ('福山通運', 52),
 ('近鉄エクスプレス', 52),
 ('ＧＭＯグローバルサイン・ホール', 52),
 ('リニカル', 51),
 ('ファンコミュニケーションズ', 50),
 ('安田倉庫', 50),
 ('セック', 49),
 ('ＧＭＯインターネット', 47),
 ('渋沢倉庫', 46),
 ('カカクコム', 43),
 ('エムティーアイ', 42),
 ('ディー・エヌ・エー', 41),
 ('パラカ', 40),
 ('三井倉庫ホールディングス', 39),
 ('東海運', 39),
 ('内外トランスライン', 37),
 ('日本石油輸送', 36),
 ('ミクシィ', 34),
 ('エン・ジャパン', 32),
 ('キャリアデザインセンター', 32),
 ('ジェイエイシーリクルートメント', 29),
 ('ＧＭＯペパボ', 29),
 ('ＬＩＦＵＬＬ', 28),
 ('アステリア', 27),
 ('サカイ引越センター', 27),
 ('エムスリー', 24),
 ('ＧＭＯペイメントゲートウェイ', 22),
 ('ぐるなび', 20),
 ('ケイヒン', 20),
 ('朝日ネット', 19),
 ('エックスネット', 18),
 ('アイフィスジャパン', 15),
 ('デジタルアーツ', 15),
 ('ヒューリック', 12),
 ('スカラ', 11),
 ('デジタルホールディングス', 11),
 ('ＵＴグループ', 11),
 ('ｅＢＡＳＥ', 11),
 ('Ｚホールディングス', 10),
 ('ドリームインキュベータ', 9),
 ('中部電力', 8)]

In [54]:
df_2008.loc[pd.IndexSlice[c.most_common()[30][0], 2008:, :], "棚卸資産"]

企業名     会計年度  四半期
ＧＭＯペパボ  2008  Q1     2,000,000.00
              Q2     2,000,000.00
              Q3     3,000,000.00
              Q4              nan
        2009  Q1              nan
              Q2              nan
              Q3     2,000,000.00
              Q4              nan
        2010  Q1              nan
              Q2              nan
              Q3     1,000,000.00
              Q4              nan
        2011  Q1              nan
              Q2              nan
              Q3     1,000,000.00
              Q4              nan
        2012  Q1              nan
              Q2              nan
              Q3              nan
              Q4              nan
        2013  Q1              nan
              Q2              nan
              Q3              nan
              Q4              nan
        2014  Q1              nan
              Q2              nan
              Q3              nan
              Q4              nan
        2015  Q1              

ほぼ欠損がない企業が結構ある。全体欠損率は約3%なので、欠損値は特定企業に集中している (まあその企業を drop すればいいから、むしろうれしい)

欠損処理は mean か ffill. 一応 meanがいいという報告はあるっぽい, 金子 (2003)
fillna(method='mean', limit=4)にしておいて、連続1年欠損があるような企業は埋めずに dropする

* 売掛金

In [159]:
# 欠損率
df_2008.loc[pd.IndexSlice[:, 2008:, :], '受取手形・売掛金／売掛金及びその他の短期債権'].isna().mean()

0.005800274491863277

欠損率自体は非常に小さい. 1%未満

In [160]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], '受取手形・売掛金／売掛金及びその他の短期債権'][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], '受取手形・売掛金／売掛金及びその他の短期債権'].isna()
]

企業名         会計年度  四半期
あみやき亭       2008  Q1    nan
                  Q2    nan
                  Q3    nan
                  Q4    nan
            2009  Q1    nan
                         ..
ＶＴホールディングス  2018  Q2    nan
                  Q3    nan
Ｚホールディングス   2013  Q1    nan
                  Q2    nan
                  Q3    nan
Name: 受取手形・売掛金／売掛金及びその他の短期債権, Length: 355, dtype: float64

In [161]:
c = collections.Counter(list(df_2008.loc[pd.IndexSlice[:, 2008:, :], '受取手形・売掛金／売掛金及びその他の短期債権'][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], '受取手形・売掛金／売掛金及びその他の短期債権'].isna()
].index.get_level_values(0)))

c.values()

dict_values([7, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 3, 1, 7, 3, 3, 3, 3, 3, 3, 15, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3])

In [162]:
c.most_common()[:10]

[('学究社', 15),
 ('あみやき亭', 7),
 ('共栄タンカー', 7),
 ('元気寿司', 6),
 ('アステリア', 4),
 ('アイシン', 3),
 ('アイティメディア', 3),
 ('アステラス製薬', 3),
 ('アドバンテスト', 3),
 ('アマダ', 3)]

1企業に欠損が集中しているわけでもない

In [165]:
df_2008.loc[pd.IndexSlice[c.most_common()[1][0], 2008:, :], '受取手形・売掛金／売掛金及びその他の短期債権']

企業名    会計年度  四半期
あみやき亭  2008  Q1              nan
             Q2              nan
             Q3              nan
             Q4              nan
       2009  Q1              nan
             Q2              nan
             Q3              nan
             Q4     4,000,000.00
       2010  Q1     4,000,000.00
             Q2     3,000,000.00
             Q3     3,000,000.00
             Q4     2,000,000.00
       2011  Q1     2,000,000.00
             Q2     1,000,000.00
             Q3     2,000,000.00
             Q4     2,000,000.00
       2012  Q1     2,000,000.00
             Q2     2,000,000.00
             Q3     2,000,000.00
             Q4     2,000,000.00
       2013  Q1     2,000,000.00
             Q2     1,000,000.00
             Q3     2,000,000.00
             Q4     1,000,000.00
       2014  Q1     3,000,000.00
             Q2     2,000,000.00
             Q3     2,000,000.00
             Q4     4,000,000.00
       2015  Q1     5,000,000.00
             Q2     3,000,

最初の方にまとめて欠損しているパターンがいくつかある. 最初の二つまでdrop するか

欠損処理は mean か ffill

* 売上総利益

In [60]:
# 欠損率
df_2008.loc[pd.IndexSlice[:, 2008:, :], ['【ＱＴＲ】売上総利益', "売上総利益［累計］"]].isna().mean()

【ＱＴＲ】売上総利益   0.01
売上総利益［累計］    0.00
dtype: float64

欠損率自体は非常に小さい. 1%未満  
'【ＱＴＲ】売上総利益'が欠損していて、"売上総利益［累計］"の差分で補填できるのであれば補填する

In [61]:
df_2008.loc[pd.IndexSlice["システムソフト", 2008:, :], ['【ＱＴＲ】売上総利益', "売上総利益［累計］"]]

【ＱＴＲ】売上総利益        売上総利益［累計］
企業名     会計年度 四半期                                
システムソフト 2008 Q1   79,000,000.00   306,000,000.00
             Q2  153,000,000.00   459,000,000.00
             Q3   99,000,000.00    99,000,000.00
             Q4   98,000,000.00   197,000,000.00
        2009 Q1             nan   184,000,000.00
             Q2             nan   317,000,000.00
             Q3   41,000,000.00    41,000,000.00
             Q4   91,000,000.00   132,000,000.00
        2010 Q1   24,000,000.00   156,000,000.00
             Q2   61,000,000.00   217,000,000.00
             Q3   27,000,000.00    27,000,000.00
             Q4   62,000,000.00    89,000,000.00
        2011 Q1   11,000,000.00   100,000,000.00
             Q2   73,000,000.00   173,000,000.00
             Q3   53,000,000.00    53,000,000.00
             Q4  126,000,000.00   179,000,000.00
        2012 Q1             nan   281,000,000.00
             Q2  172,000,000.00   453,000,000.00
             Q3  149,000,000.00   149,000,000.00
             Q4  366,000,000.00   515,000,000.00
        2013 Q1  256,000,000.00   771,000,000.00
             Q2  331,000,000.00 1,102,000,000.00
             Q3  306,000,000.00   306,000,000.00
             Q4  372,000,000.00   678,000,000.00
        2014 Q1  283,000,000.00   961,000,000.00
             Q2  294,000,000.00 1,255,000,000.00
             Q3  203,000,000.00   203,000,000.00
             Q4  283,000,000.00   486,000,000.00
        2015 Q1  209,000,000.00   695,000,000.00
             Q2  440,000,000.00 1,135,000,000.00
             Q3  214,000,000.00   214,000,000.00
             Q4  284,000,000.00   498,000,000.00
        2016 Q1  297,000,000.00   795,000,000.00
             Q2  381,000,000.00 1,176,000,000.00
             Q3  172,000,000.00   172,000,000.00
             Q4  349,000,000.00   521,000,000.00
        2017 Q1  177,000,000.00   698,000,000.00
             Q2  204,000,000.00   902,000,000.00
             Q3             nan   100,000,000.00
             Q4  124,000,000.00   224,000,000.00
        2018 Q1   53,000,000.00   277,000,000.00
             Q2  163,000,000.00   440,000,000.00
             Q3  130,000,000.00   130,000,000.00
             Q4  128,000,000.00   258,000,000.00
        2019 Q1   93,000,000.00   351,000,000.00
             Q2  412,000,000.00   763,000,000.00
             Q3   32,000,000.00    32,000,000.00
             Q4   82,000,000.00   114,000,000.00
        2020 Q1  137,000,000.00   251,000,000.00
             Q2   88,000,000.00   339,000,000.00
             Q3  419,000,000.00   419,000,000.00
             Q4  359,000,000.00   778,000,000.00

でもよく見ると、この会社はQ3が類型と3ヶ月が一致、つまり12月決算かも?  
--> 6, 9, 12, 3に四半期報告する会社は必ずしも 3月決算ではない  
--> 3月決算に絞るにしても、累計の欠損処理するにしても、会社の決算月(もしくは決算種別フラグ?)のデータが必要!

---
決算種別フラグから、2008年度以降 3月本決算企業とそれ以外(6, 9, 12)を特定  
もしかすると途中で本決算月を変えてるかもしれない

In [68]:
df_2008["決算種別フラグ"].unique()

array([10, 21, 22, 23, 24])

In [85]:
# 3月のレコード数
df_2008[df_2008["決算期（月）"] == 3].shape[0]

30412

In [92]:
# 3月が本決算であるレコード数
df_2008[(df_2008["決算期（月）"] == 3) & (df_2008["決算種別フラグ"] == 10)].shape[0]

26608

In [93]:
# 3月がQ1決算であるレコード数
df_2008[(df_2008["決算期（月）"] == 3) & (df_2008["決算種別フラグ"] == 21)].shape[0]

2447

In [94]:
# 3月がQ2決算であるレコード数
df_2008[(df_2008["決算期（月）"] == 3) & (df_2008["決算種別フラグ"] == 22)].shape[0]

845

In [95]:
# 3月がQ3決算であるレコード数
df_2008[(df_2008["決算期（月）"] == 3) & (df_2008["決算種別フラグ"] == 23)].shape[0]

511

In [96]:
# 3月がQ4決算であるレコード数
df_2008[(df_2008["決算期（月）"] == 3) & (df_2008["決算種別フラグ"] == 24)].shape[0]

1

In [113]:
# 少なくとも2008年度以降 1レコード3月本決算がある企業

df_2008.loc[pd.IndexSlice[:, 2008:, "Q4"], :][
    df_2008.loc[pd.IndexSlice[:, 2008:, "Q4"], "決算種別フラグ"] == 10
].index.get_level_values(0).unique()

Index(['あみやき亭', 'あらた', 'いすゞ自動車', 'いなげや', 'かどや製油', 'きんでん', 'ぐるなび', 'さくらインターネット',
       'たけびし', 'なとり',
       ...
       'ＴＹＫ', 'ＵＡＣＪ', 'ＵＴグループ', 'ＶＴホールディングス', 'ＷＤＢホールディングス', 'ＷＯＷ　ＷＯＲＬＤ',
       'ＷＯＷＯＷ', 'Ｚホールディングス', 'ＺＯＺＯ', 'ｅＢＡＳＥ'],
      dtype='object', name='企業名', length=1246)

このうち、2008/06 ~ 2020\03までの13年度中すべてが3月本決算である企業を特定する

In [142]:
# Q4 が 本決算である企業別割合が100%である企業
mar = (df_2008.loc[pd.IndexSlice[:, 2008:, "Q4"], "決算種別フラグ"] == 10).mean(level=0)[
    (df_2008.loc[pd.IndexSlice[:, 2008:, "Q4"], "決算種別フラグ"] == 10).mean(level=0) == 1
].index.get_level_values(0)

mar


Index(['あみやき亭', 'あらた', 'いすゞ自動車', 'いなげや', 'かどや製油', 'きんでん', 'ぐるなび', 'さくらインターネット',
       'たけびし', 'なとり',
       ...
       'ＴＹＫ', 'ＵＡＣＪ', 'ＵＴグループ', 'ＶＴホールディングス', 'ＷＤＢホールディングス', 'ＷＯＷ　ＷＯＲＬＤ',
       'ＷＯＷＯＷ', 'Ｚホールディングス', 'ＺＯＺＯ', 'ｅＢＡＳＥ'],
      dtype='object', name='企業名', length=1177)

2008/06 ~ 2020\03までの13年度中すべてが3月本決算である企業は この1177 社

In [144]:
df_2008 = df_2008.loc[pd.IndexSlice[mar, :, :], :]

df_2008

決算期（年）  決算期（月）      決算期  連結基準フラグ  決算種別フラグ       決算発表日  \
企業名   会計年度 四半期                                                          
あみやき亭 1999 Q4     2000       3  2000/03        0       10         NaN   
      2000 Q4     2001       3  2001/03        0       10         NaN   
      2001 Q2     2001       9  2001/09        0       22         NaN   
           Q4     2002       3  2002/03        0       10         NaN   
      2002 Q2     2002       9  2002/09        0       22         NaN   
...                ...     ...      ...      ...      ...         ...   
ｅＢＡＳＥ 2019 Q4     2020       3  2020/03        1       10  2020-05-15   
      2020 Q1     2020       6  2020/06        1       21  2020-07-31   
           Q2     2020       9  2020/09        1       22  2020-10-30   
           Q3     2020      12  2020/12        1       23  2021-01-29   
           Q4     2021       3  2021/03        1       10  2021-05-14   

               事業年度開始年月日［３ヵ月］   事業年度終了年月日  １株当たり利益［３ヵ月］  １株当たり利益［累計］  \
企業名   会計年度 四半期                                                         
あみやき亭 1999 Q4             NaN  2000-03-31           nan   129,967.34   
      2000 Q4             NaN  2001-03-31           nan   156,604.77   
      2001 Q2             NaN  2001-09-30           nan    70,231.40   
           Q4             NaN  2002-03-31           nan   112,684.36   
      2002 Q2             NaN  2002-09-30           nan    28,525.07   
...                       ...         ...           ...          ...   
ｅＢＡＳＥ 2019 Q4      2020-01-01  2020-03-31           nan          nan   
      2020 Q1      2020-04-01  2020-06-30          2.01         2.01   
           Q2      2020-07-01  2020-09-30          3.81         5.82   
           Q3      2020-10-01  2020-12-31          3.07         8.89   
           Q4             NaN  2021-03-31           nan        18.90   

                当期純利益（連結）［累計］      【ＱＴＲ】当期利益          棚卸資産  \
企業名   会計年度 四半期                                               
あみやき亭 1999 Q4             nan            nan  4,000,000.00   
      2000 Q4             nan            nan  7,000,000.00   
      2001 Q2             nan            nan  7,000,000.00   
           Q4             nan            nan  8,000,000.00   
      2002 Q2             nan            nan 10,000,000.00   
...                       ...            ...           ...   
ｅＢＡＳＥ 2019 Q4  904,000,000.00 440,000,000.00  1,000,000.00   
      2020 Q1   92,000,000.00  92,000,000.00  4,000,000.00   
           Q2  267,000,000.00 175,000,000.00  4,000,000.00   
           Q3  409,000,000.00 142,000,000.00  5,000,000.00   
           Q4  869,000,000.00 460,000,000.00           nan   

                受取手形・売掛金／売掛金及びその他の短期債権          資本的支出        売上総利益［累計］  \
企業名   会計年度 四半期                                                           
あみやき亭 1999 Q4                      nan 151,500,000.00 1,219,000,000.00   
      2000 Q4                      nan 151,500,000.00 2,056,000,000.00   
      2001 Q2                      nan 185,000,000.00 1,382,000,000.00   
           Q4                      nan 520,000,000.00 2,801,000,000.00   
      2002 Q2                      nan 261,000,000.00 1,716,000,000.00   
...                                ...            ...              ...   
ｅＢＡＳＥ 2019 Q4           965,000,000.00  15,500,000.00 2,326,000,000.00   
      2020 Q1           595,000,000.00   7,000,000.00   390,000,000.00   
           Q2           531,000,000.00   7,000,000.00   906,000,000.00   
           Q3           541,000,000.00  14,500,000.00 1,375,000,000.00   
           Q4           844,000,000.00  14,500,000.00 2,289,000,000.00   

                   【ＱＴＲ】売上総利益   販売費及び一般管理費［累計］       法人税等［累計］  \
企業名   会計年度 四半期                                                  
あみやき亭 1999 Q4             nan   932,000,000.00 130,000,000.00   
      2000 Q4             nan 1,544,000,000.00 225,000,000.00   
      2001 Q2             nan 1,065,000,000.00 121,000,000.00   
           Q4             nan 2,24

---

'【ＱＴＲ】売上総利益' を "売上総利益［累計］"の差分で補填できるところは補填する

というか累計をすべて差分したほうがいいかも

In [187]:
df_2008[["【ＱＴＲ】売上総利益", "売上総利益［累計］"]][(df_2008["【ＱＴＲ】売上総利益"].notna()) & (df_2008["売上総利益［累計］"].isna())]

,,,【ＱＴＲ】売上総利益,売上総利益［累計］
企業名,会計年度,四半期,,


"【ＱＴＲ】売上総利益"あり、 "売上総利益［累計］"なし、というレコードはないから

In [201]:
# 欠損状況
df_2008.loc[pd.IndexSlice[:, 2008:, :], "売上総利益［累計］"][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], "売上総利益［累計］"].isna()
]

企業名    会計年度  四半期
カカクコム  2016  Q1    nan
             Q2    nan
             Q3    nan
       2017  Q1    nan
             Q2    nan
                    ..
ＫＤＤＩ   2012  Q4    nan
       2013  Q1    nan
             Q2    nan
             Q3    nan
             Q4    nan
Name: 売上総利益［累計］, Length: 104, dtype: float64

In [202]:
c = collections.Counter(list(df_2008.loc[pd.IndexSlice[:, 2008:, :], "売上総利益［累計］"][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], "売上総利益［累計］"].isna()
].index.get_level_values(0)))

c.values()

dict_values([15, 1, 1, 1, 1, 16, 45, 24])

In [203]:
c.most_common()

[('ＨＯＹＡ', 45),
 ('ＫＤＤＩ', 24),
 ('日本電信電話', 16),
 ('カカクコム', 15),
 ('トリドールホールディングス', 1),
 ('モリテック\u3000スチール', 1),
 ('富士ピー・エス', 1),
 ('日本ライフライン', 1)]

特定の企業に欠損が集中 (drop)

In [208]:
df_2008.loc[pd.IndexSlice[c.most_common()[4][0], 2008:, :], ['【ＱＴＲ】売上総利益', "売上総利益［累計］"]]

【ＱＴＲ】売上総利益          売上総利益［累計］
企業名           会計年度 四半期                                     
トリドールホールディングス 2008 Q1   3,818,000,000.00   3,818,000,000.00
                   Q2   4,399,000,000.00   8,217,000,000.00
                   Q3   4,739,000,000.00  12,956,000,000.00
                   Q4   5,280,000,000.00  18,236,000,000.00
              2009 Q1   6,272,000,000.00   6,272,000,000.00
                   Q2   7,487,000,000.00  13,759,000,000.00
                   Q3   7,521,000,000.00  21,280,000,000.00
                   Q4   7,886,000,000.00  29,166,000,000.00
              2010 Q1   8,519,000,000.00   8,519,000,000.00
                   Q2                nan                nan
                   Q3                nan  27,268,000,000.00
                   Q4   9,141,000,000.00  36,409,000,000.00
              2011 Q1  10,609,000,000.00  10,609,000,000.00
                   Q2  11,600,000,000.00  22,209,000,000.00
                   Q3  11,603,000,000.00  33,812,000,000.00
                   Q4  11,530,000,000.00  45,342,000,000.00
              2012 Q1  12,576,000,000.00  12,576,000,000.00
                   Q2  13,736,000,000.00  26,312,000,000.00
                   Q3  13,045,000,000.00  39,357,000,000.00
                   Q4  13,927,000,000.00  53,284,000,000.00
              2013 Q1  14,651,000,000.00  14,651,000,000.00
                   Q2  15,419,000,000.00  30,070,000,000.00
                   Q3  14,463,000,000.00  44,533,000,000.00
                   Q4                nan  59,411,000,000.00
              2014 Q1  15,086,000,000.00  15,086,000,000.00
                   Q2  17,135,000,000.00  32,221,000,000.00
                   Q3  16,708,000,000.00  48,929,000,000.00
                   Q4  16,057,000,000.00  64,986,000,000.00
              2015 Q1  17,329,000,000.00  17,329,000,000.00
                   Q2  18,448,000,000.00  35,777,000,000.00
                   Q3  17,993,000,000.00  53,770,000,000.00
                   Q4  17,712,000,000.00  71,482,000,000.00
              2016 Q1  17,935,000,000.00  17,935,000,000.00
                   Q2  19,429,000,000.00  37,364,000,000.00
                   Q3  19,449,000,000.00  56,813,000,000.00
                   Q4  18,750,000,000.00  75,563,000,000.00
              2017 Q1  20,237,000,000.00  20,237,000,000.00
                   Q2  21,398,000,000.00  41,635,000,000.00
                   Q3  20,862,000,000.00  62,497,000,000.00
                   Q4  23,147,000,000.00  85,644,000,000.00
              2018 Q1  25,834,000,000.00  25,834,000,000.00
                   Q2  26,728,000,000.00  52,562,000,000.00
                   Q3  26,980,000,000.00  79,542,000,000.00
                   Q4  26,362,000,000.00 105,904,000,000.00
              2019 Q1  28,836,000,000.00  28,836,000,000.00
                   Q2  30,279,000,000.00  59,115,000,000.00
                   Q3  29,420,000,000.00  88,535,000,000.00
                   Q4  27,738,000,000.00 116,273,000,000.00
              2020 Q1  20,165,000,000.00  20,165,000,000.00
                   Q2  26,760,000,000.00  46,925,000,000.00
                   Q3  27,802,000,000.00  74,727,000,000.00
                   Q4  25,304,000,000.00 100,031,000,000.00

このあたりくらいまでは dropするか  
つまり、fillna(method='mean', limit=1)にしておいて、連続1年欠損があるような企業は埋めずに dropする

In [188]:
# 一企業例
df_2008.loc[pd.IndexSlice["トリドールホールディングス", 2008:, :], ["【ＱＴＲ】売上総利益", "売上総利益［累計］"]]

【ＱＴＲ】売上総利益          売上総利益［累計］
企業名           会計年度 四半期                                     
トリドールホールディングス 2008 Q1   3,818,000,000.00   3,818,000,000.00
                   Q2   4,399,000,000.00   8,217,000,000.00
                   Q3   4,739,000,000.00  12,956,000,000.00
                   Q4   5,280,000,000.00  18,236,000,000.00
              2009 Q1   6,272,000,000.00   6,272,000,000.00
                   Q2   7,487,000,000.00  13,759,000,000.00
                   Q3   7,521,000,000.00  21,280,000,000.00
                   Q4   7,886,000,000.00  29,166,000,000.00
              2010 Q1   8,519,000,000.00   8,519,000,000.00
                   Q2                nan                nan
                   Q3                nan  27,268,000,000.00
                   Q4   9,141,000,000.00  36,409,000,000.00
              2011 Q1  10,609,000,000.00  10,609,000,000.00
                   Q2  11,600,000,000.00  22,209,000,000.00
                   Q3  11,603,000,000.00  33,812,000,000.00
                   Q4  11,530,000,000.00  45,342,000,000.00
              2012 Q1  12,576,000,000.00  12,576,000,000.00
                   Q2  13,736,000,000.00  26,312,000,000.00
                   Q3  13,045,000,000.00  39,357,000,000.00
                   Q4  13,927,000,000.00  53,284,000,000.00
              2013 Q1  14,651,000,000.00  14,651,000,000.00
                   Q2  15,419,000,000.00  30,070,000,000.00
                   Q3  14,463,000,000.00  44,533,000,000.00
                   Q4                nan  59,411,000,000.00
              2014 Q1  15,086,000,000.00  15,086,000,000.00
                   Q2  17,135,000,000.00  32,221,000,000.00
                   Q3  16,708,000,000.00  48,929,000,000.00
                   Q4  16,057,000,000.00  64,986,000,000.00
              2015 Q1  17,329,000,000.00  17,329,000,000.00
                   Q2  18,448,000,000.00  35,777,000,000.00
                   Q3  17,993,000,000.00  53,770,000,000.00
                   Q4  17,712,000,000.00  71,482,000,000.00
              2016 Q1  17,935,000,000.00  17,935,000,000.00
                   Q2  19,429,000,000.00  37,364,000,000.00
                   Q3  19,449,000,000.00  56,813,000,000.00
                   Q4  18,750,000,000.00  75,563,000,000.00
              2017 Q1  20,237,000,000.00  20,237,000,000.00
                   Q2  21,398,000,000.00  41,635,000,000.00
                   Q3  20,862,000,000.00  62,497,000,000.00
                   Q4  23,147,000,000.00  85,644,000,000.00
              2018 Q1  25,834,000,000.00  25,834,000,000.00
                   Q2  26,728,000,000.00  52,562,000,000.00
                   Q3  26,980,000,000.00  79,542,000,000.00
                   Q4  26,362,000,000.00 105,904,000,000.00
              2019 Q1  28,836,000,000.00  28,836,000,000.00
                   Q2  30,279,000,000.00  59,115,000,000.00
                   Q3  29,420,000,000.00  88,535,000,000.00
                   Q4  27,738,000,000.00 116,273,000,000.00
              2020 Q1  20,165,000,000.00  20,165,000,000.00
                   Q2  26,760,000,000.00  46,925,000,000.00
                   Q3  27,802,000,000.00  74,727,000,000.00
                   Q4  25,304,000,000.00 100,031,000,000.00

In [191]:
df_2008_copy = df_2008.copy()
isna_series = df_2008_copy.loc[pd.IndexSlice["トリドールホールディングス", :, :], "売上総利益［累計］"][
    df_2008_copy.loc[pd.IndexSlice["トリドールホールディングス", :, :], "売上総利益［累計］"].isna()
]

isna_series

企業名            会計年度  四半期
トリドールホールディングス  2010  Q2    nan
Name: 売上総利益［累計］, dtype: float64

In [186]:
"トリドールホールディングス", 2008:, :], ["【ＱＴＲ】売上総利益", "売上総利益［累計］"


isna_series = df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], "資本的支出"].isna()

for i in range(len(isna_series)-1):
    if (isna_series.iloc[i] == True) & (isna_series.iloc[i+1] == False): 
        # i が nan, i+1 が nan じゃないとき、
        ind_non_nan = df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], :].index[i+1]
        print("index of record be divided: ", ind_non_nan)
        print(df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], "資本的支出"][i], df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], "資本的支出"][i+1])
        num_nan = 0
        ind_nan_list = []
        while isna_series.iloc[i - num_nan]:
            # nan がどのくらい続いていたのかカウントして、
            num_nan += 1
            ind_nan_list.append(df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], "資本的支出"].index[i+1 - num_nan])
        print("num nan: ", num_nan)
        print("index of nan: ", ind_nan_list)
        # nan じゃない i+1 を num_nan + 1の分だけ割っておいて、
        divided_value = (df_2008_copy.loc[pd.IndexSlice[ind_non_nan[0], ind_non_nan[1], ind_non_nan[2]], "資本的支出"] / (num_nan+1))
        print("divided value: ", divided_value)
        df_2008_copy.loc[pd.IndexSlice[ind_non_nan[0], ind_non_nan[1], ind_non_nan[2]], "資本的支出"] = divided_value
        for ind_nan in ind_nan_list:
            # nan にも同じ割った値を代入
            df_2008_copy.loc[pd.IndexSlice[ind_nan[0], ind_nan[1], ind_nan[2]], "資本的支出"] = divided_value

【ＱＴＲ】売上総利益          売上総利益［累計］
企業名           会計年度 四半期                                     
トリドールホールディングス 2008 Q1   3,818,000,000.00   3,818,000,000.00
                   Q2   4,399,000,000.00   8,217,000,000.00
                   Q3   4,739,000,000.00  12,956,000,000.00
                   Q4   5,280,000,000.00  18,236,000,000.00
              2009 Q1   6,272,000,000.00   6,272,000,000.00
                   Q2   7,487,000,000.00  13,759,000,000.00
                   Q3   7,521,000,000.00  21,280,000,000.00
                   Q4   7,886,000,000.00  29,166,000,000.00
              2010 Q1   8,519,000,000.00   8,519,000,000.00
                   Q2                nan                nan
                   Q3                nan  27,268,000,000.00
                   Q4   9,141,000,000.00  36,409,000,000.00
              2011 Q1  10,609,000,000.00  10,609,000,000.00
                   Q2  11,600,000,000.00  22,209,000,000.00
                   Q3  11,603,000,000.00  33,812,000,000.00
                   Q4  11,530,000,000.00  45,342,000,000.00
              2012 Q1  12,576,000,000.00  12,576,000,000.00
                   Q2  13,736,000,000.00  26,312,000,000.00
                   Q3  13,045,000,000.00  39,357,000,000.00
                   Q4  13,927,000,000.00  53,284,000,000.00
              2013 Q1  14,651,000,000.00  14,651,000,000.00
                   Q2  15,419,000,000.00  30,070,000,000.00
                   Q3  14,463,000,000.00  44,533,000,000.00
                   Q4                nan  59,411,000,000.00
              2014 Q1  15,086,000,000.00  15,086,000,000.00
                   Q2  17,135,000,000.00  32,221,000,000.00
                   Q3  16,708,000,000.00  48,929,000,000.00
                   Q4  16,057,000,000.00  64,986,000,000.00
              2015 Q1  17,329,000,000.00  17,329,000,000.00
                   Q2  18,448,000,000.00  35,777,000,000.00
                   Q3  17,993,000,000.00  53,770,000,000.00
                   Q4  17,712,000,000.00  71,482,000,000.00
              2016 Q1  17,935,000,000.00  17,935,000,000.00
                   Q2  19,429,000,000.00  37,364,000,000.00
                   Q3  19,449,000,000.00  56,813,000,000.00
                   Q4  18,750,000,000.00  75,563,000,000.00
              2017 Q1  20,237,000,000.00  20,237,000,000.00
                   Q2  21,398,000,000.00  41,635,000,000.00
                   Q3  20,862,000,000.00  62,497,000,000.00
                   Q4  23,147,000,000.00  85,644,000,000.00
              2018 Q1  25,834,000,000.00  25,834,000,000.00
                   Q2  26,728,000,000.00  52,562,000,000.00
                   Q3  26,980,000,000.00  79,542,000,000.00
                   Q4  26,362,000,000.00 105,904,000,000.00
              2019 Q1  28,836,000,000.00  28,836,000,000.00
                   Q2  30,279,000,000.00  59,115,000,000.00
                   Q3  29,420,000,000.00  88,535,000,000.00
                   Q4  27,738,000,000.00 116,273,000,000.00
              2020 Q1  20,165,000,000.00  20,165,000,000.00
                   Q2  26,760,000,000.00  46,925,000,000.00
                   Q3  27,802,000,000.00  74,727,000,000.00
                   Q4  25,304,000,000.00 100,031,000,000.00

* 販売費及び一般管理費［累計］

In [209]:
# 欠損率
df_2008.loc[pd.IndexSlice[:, 2008:, :], ['販売費及び一般管理費［累計］']].isna().mean()

販売費及び一般管理費［累計］   0.01
dtype: float64

In [210]:
# 欠損状況
df_2008.loc[pd.IndexSlice[:, 2008:, :], '販売費及び一般管理費［累計］'][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], '販売費及び一般管理費［累計］'].isna()
]

企業名            会計年度  四半期
トリドールホールディングス  2010  Q2    nan
モリテック　スチール     2011  Q4    nan
中国電力           2008  Q1    nan
                     Q2    nan
                     Q3    nan
                            ..
ＨＯＹＡ           2019  Q4    nan
               2020  Q1    nan
                     Q2    nan
                     Q3    nan
                     Q4    nan
Name: 販売費及び一般管理費［累計］, Length: 555, dtype: float64

In [211]:
c = collections.Counter(list(df_2008.loc[pd.IndexSlice[:, 2008:, :], '販売費及び一般管理費［累計］'][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], '販売費及び一般管理費［累計］'].isna()
].index.get_level_values(0)))

c.values()

dict_values([1, 1, 52, 52, 52, 52, 52, 52, 1, 1, 16, 52, 21, 52, 52, 1, 45])

In [212]:
c.most_common()

[('中国電力', 52),
 ('中部電力', 52),
 ('九州電力', 52),
 ('北海道電力', 52),
 ('北陸電力', 52),
 ('四国電力', 52),
 ('東京電力ホールディングス', 52),
 ('沖縄電力', 52),
 ('関西電力', 52),
 ('ＨＯＹＡ', 45),
 ('東北電力', 21),
 ('日本電信電話', 16),
 ('トリドールホールディングス', 1),
 ('モリテック\u3000スチール', 1),
 ('富士ピー・エス', 1),
 ('日本ライフライン', 1),
 ('電源開発', 1)]

特定の企業に欠損が集中 (drop)

In [217]:
df_2008.loc[pd.IndexSlice[c.most_common()[11][0], 2008:, :], ['販売費及び一般管理費［累計］']]

販売費及び一般管理費［累計］
企業名    会計年度 四半期                     
日本電信電話 2008 Q1  1,219,207,000,000.00
            Q2  2,453,577,000,000.00
            Q3  3,747,314,000,000.00
            Q4  5,145,883,000,000.00
       2009 Q1  1,198,565,000,000.00
            Q2  2,424,889,000,000.00
            Q3  3,679,405,000,000.00
            Q4  5,020,932,000,000.00
       2010 Q1  1,203,113,000,000.00
            Q2  2,358,454,000,000.00
            Q3  3,568,402,000,000.00
            Q4  4,956,215,000,000.00
       2011 Q1  1,182,553,000,000.00
            Q2  2,398,247,000,000.00
            Q3  3,592,136,000,000.00
            Q4  4,907,457,000,000.00
       2012 Q1  1,178,593,000,000.00
            Q2  2,394,129,000,000.00
            Q3  3,595,124,000,000.00
            Q4  4,928,590,000,000.00
       2013 Q1  1,168,862,000,000.00
            Q2  2,356,505,000,000.00
            Q3  3,525,678,000,000.00
            Q4  4,821,329,000,000.00
       2014 Q1  1,171,094,000,000.00
            Q2  2,327,096,000,000.00
            Q3  3,504,967,000,000.00
            Q4  4,726,659,000,000.00
       2015 Q1  1,119,289,000,000.00
            Q2  2,240,799,000,000.00
            Q3  3,358,241,000,000.00
            Q4  4,566,807,000,000.00
       2016 Q1  1,005,091,000,000.00
            Q2  2,051,757,000,000.00
            Q3  3,139,184,000,000.00
            Q4  4,322,907,000,000.00
       2017 Q1                   nan
            Q2                   nan
            Q3                   nan
            Q4                   nan
       2018 Q1                   nan
            Q2                   nan
            Q3                   nan
            Q4                   nan
       2019 Q1                   nan
            Q2                   nan
            Q3                   nan
            Q4                   nan
       2020 Q1                   nan
            Q2                   nan
            Q3                   nan
            Q4                   nan

このあたりくらいまでは dropするか  
つまり、fillna(method='mean', limit=1)にしておいて、連続1年欠損があるような企業は埋めずに dropする

* 法人税

In [221]:
# 欠損率
df_2008.loc[pd.IndexSlice[:, 2008:, :], ['法人税等［累計］', '法人税・住民税及び事業税合計／当期税金費用［累計］']].isna().mean()

法人税等［累計］                    0.00
法人税・住民税及び事業税合計／当期税金費用［累計］   0.27
dtype: float64

In [37]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], ['法人税等［累計］', '法人税・住民税及び事業税合計／当期税金費用［累計］']][
    (df_2008.loc[pd.IndexSlice[:, 2008:, :], '法人税等［累計］'].isna()) & 
    (df_2008.loc[pd.IndexSlice[:, 2008:, :], '法人税・住民税及び事業税合計／当期税金費用［累計］'].notna())
]

法人税等［累計］  法人税・住民税及び事業税合計／当期税金費用［累計］
企業名             会計年度 四半期                                     
アイ・オー・データ機器     2019 Q1        nan             916,000,000.00
アツギ             2019 Q2        nan              91,000,000.00
アドウェイズ          2008 Q3        nan               4,000,000.00
アルプス技研          2011 Q2        nan             106,000,000.00
ウェルネット          2010 Q2        nan               5,000,000.00
ウッドワン           2016 Q1        nan             205,000,000.00
エコナックホールディングス   2013 Q2        nan               1,000,000.00
                2015 Q4        nan               3,000,000.00
                2017 Q2        nan               6,000,000.00
                     Q3        nan               7,000,000.00
                2020 Q1        nan               1,000,000.00
エスリード           2011 Q1        nan              28,000,000.00
エノモト            2015 Q1        nan              45,000,000.00
エンシュウ           2011 Q1        nan               8,000,000.00
クイック            2010 Q1        nan               2,000,000.00
グリムス            2012 Q1        nan               6,000,000.00
サンフロンティア不動産     2013 Q1        nan              24,000,000.00
ジャパンベストレスキューシステ 2009 Q3        nan               2,000,000.00
ダイドーリミテッド       2019 Q1        nan              42,000,000.00
ダントーホールディングス    2016 Q3        nan              25,000,000.00
ナカヨ             2020 Q3        nan              51,000,000.00
バリューコマース        2009 Q4        nan               1,000,000.00
ヒノキヤグループ        2019 Q4        nan             170,000,000.00
フコク             2020 Q2        nan             194,000,000.00
ペガサスミシン製造       2008 Q1        nan              76,000,000.00
メンバーズ           2010 Q2        nan               1,000,000.00
ユニカフェ           2008 Q3        nan               1,000,000.00
ラサ工業            2009 Q2        nan              59,000,000.00
丸運              2014 Q1        nan              16,000,000.00
佐藤商事            2009 Q1        nan              10,000,000.00
富士興産            2015 Q2        nan              16,000,000.00
山一電機            2009 Q3        nan              96,000,000.00
岩崎通信機           2014 Q1        nan               9,000,000.00
川西倉庫            2009 Q1        nan               7,000,000.00
新日本空調           2008 Q2        nan              71,000,000.00
日本曹達            2020 Q3        nan             861,000,000.00
日本通信            2016 Q1        nan               1,000,000.00
                     Q2        nan               2,000,000.00
日本電子材料          2012 Q1        nan              44,000,000.00
明和地所            2019 Q1        nan               5,000,000.00
明治海運            2020 Q3        nan             459,000,000.00
星和電機            2011 Q1        nan               5,000,000.00
木曽路             2014 Q3        nan              87,000,000.00
本多通信工業          2008 Q1        nan              10,000,000.00
東京テアトル          2014 Q1        nan              32,000,000.00
東洋シヤッター         2011 Q2        nan              26,000,000.00
東海染工            2011 Q3        nan              32,000,000.00
椿本興業            2009 Q2        nan              65,000,000.00
極東貿易            2010 Q1        nan              21,000,000.00
池上通信機           2008 Q2        nan              10,000,000.00
盟和産業            2014 Q4        nan              97,000,000.00
石川製作所           2009 Q1        nan               8,000,000.00
                     Q2        nan               5,000,000.00
                     Q3        nan               4,000,000.00
神戸電鉄            2020 Q1        nan               4,000,000.00
秀英予備校           2016 Q1        nan              13,000,000.00
第一屋製パン          2012 Q4        nan              31,000,000.00
藤倉コンポジット        2009 Q2        nan              96,000,000.00
藤倉化成            2012 Q4        nan             560,000,000.00
駒井ハルテック         2012 Q3        nan              21,000,000.00
                2013 Q1        nan               6,000,000.00
高田機工            2008 Q2        nan              10,000,000.00
ＣＥホールディングス      2010 Q3        nan               1,000,000

'法人税等［累計］'が欠損して入れも'法人税・住民税及び事業税合計／当期税金費用［累計］'があるレコードもあるから  
'法人税等［累計］'をベースにして、欠損を'法人税・住民税及び事業税合計／当期税金費用［累計］'を代入して補填可能  
--> '法人税等［累計］' !='法人税・住民税及び事業税合計／当期税金費用［累計］' がほとんどなので、やっぱり補填しない方がいいかも

In [38]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], ['法人税等［累計］', '法人税・住民税及び事業税合計／当期税金費用［累計］']][
    (df_2008.loc[pd.IndexSlice[:, 2008:, :], '法人税等［累計］'].isna()) & 
    (df_2008.loc[pd.IndexSlice[:, 2008:, :], '法人税・住民税及び事業税合計／当期税金費用［累計］'].isna())
]

法人税等［累計］  法人税・住民税及び事業税合計／当期税金費用［累計］
企業名             会計年度 四半期                                     
アイティメディア        2014 Q1        nan                        nan
アクセル            2019 Q1        nan                        nan
アステリア           2010 Q1        nan                        nan
                2011 Q1        nan                        nan
                2012 Q1        nan                        nan
アドソル日進          2009 Q1        nan                        nan
                2010 Q1        nan                        nan
                2011 Q1        nan                        nan
                2012 Q1        nan                        nan
インテリジェント　ウェイブ   2012 Q4        nan                        nan
ウェルネット          2009 Q1        nan                        nan
エコナックホールディングス   2010 Q1        nan                        nan
                2013 Q1        nan                        nan
                2014 Q1        nan                        nan
                2015 Q1        nan                        nan
                2016 Q1        nan                        nan
                2018 Q1        nan                        nan
                2020 Q2        nan                        nan
エステールホールディングス   2019 Q1        nan                        nan
オルガノ            2016 Q1        nan                        nan
キムラタン           2015 Q1        nan                        nan
                2016 Q1        nan                        nan
                2017 Q1        nan                        nan
                2019 Q1        nan                        nan
キャリアデザインセンター    2009 Q3        nan                        nan
                2010 Q3        nan                        nan
                     Q4        nan                        nan
                2011 Q3        nan                        nan
クボテック           2008 Q1        nan                        nan
                2019 Q1        nan                        nan
                2020 Q2        nan                        nan
コムチュア           2010 Q1        nan                        nan
サニーサイドアップグループ   2009 Q2        nan                        nan
                     Q4        nan                        nan
                2010 Q1        nan                        nan
システムソフト         2016 Q3        nan                        nan
スターティアホールディングス  2010 Q1        nan                        nan
ソースネクスト         2009 Q1        nan                        nan
                2010 Q1        nan                        nan
                2011 Q1        nan                        nan
チタン工業           2016 Q1        nan                        nan
                2017 Q1        nan                        nan
ツカダ・グローバルホールディン 2017 Q4        nan                        nan
ティアック           2016 Q1        nan                        nan
                2019 Q2        nan                        nan
ディア・ライフ         2009 Q3        nan                        nan
                     Q4        nan                        nan
                2010 Q1        nan                        nan
                     Q2        nan                        nan
デジタルガレージ        2010 Q2        nan                        nan
トリドールホールディングス   2010 Q2        nan                        nan
ピー・シー・エー        2011 Q1        nan                        nan
フィールズ           2019 Q2        nan                        nan
フォーカスシステムズ      2011 Q1        nan                        nan
                     Q2        nan                        nan
                2012 Q2        nan                        nan
                     Q3        nan                        nan
                2013 Q1        nan                        nan
フジオフードグループ本社    2008 Q3        nan                        nan
マーベラス           2009 Q1        nan                        nan
メンバーズ           2009 Q1        nan                        nan
                2010 Q1        nan                        nan
モリテック　スチール      2011 Q4        nan                        

それでも両方とも欠損してることもあるので、最終的にはmean, ffillか、dropか

* 税引前利益

In [39]:
# 欠損状況
df_2008.loc[pd.IndexSlice[:, 2008:, :], [
    '税金等調整前当期純利益［累計］', 
    '税金等調整前当期利益［累計］',
    '【ＱＴＲ】税金等調整前当期利益', 
    '１株当たり税引前利益']
].isna().mean()

税金等調整前当期純利益［累計］   0.00
税金等調整前当期利益［累計］    0.04
【ＱＴＲ】税金等調整前当期利益   0.04
１株当たり税引前利益        0.01
dtype: float64

In [42]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], [
    '税金等調整前当期純利益［累計］', 
    '税金等調整前当期利益［累計］',
    '【ＱＴＲ】税金等調整前当期利益', 
    '１株当たり税引前利益']][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], '税金等調整前当期純利益［累計］'].isna()
]

税金等調整前当期純利益［累計］  税金等調整前当期利益［累計］  【ＱＴＲ】税金等調整前当期利益  \
企業名            会計年度 四半期                                                     
ウェルネット         2009 Q1               nan             nan              nan   
               2010 Q2               nan             nan              nan   
システムソフト        2010 Q1               nan             nan              nan   
ジャムコ           2008 Q2               nan             nan              nan   
スターティアホールディングス 2009 Q3               nan             nan              nan   
ティラド           2009 Q4               nan             nan              nan   
トリドールホールディングス  2010 Q2               nan             nan              nan   
ノーリツ鋼機         2011 Q1               nan             nan              nan   
フォーカスシステムズ     2013 Q1               nan             nan              nan   
フジオフードグループ本社   2008 Q3               nan             nan              nan   
ペッパーフードサービス    2012 Q1               nan             nan              nan   
モリテック　スチール     2011 Q4               nan             nan              nan   
協栄産業           2012 Q3               nan             nan              nan   
大豊建設           2013 Q1               nan             nan              nan   
大阪チタニウムテクノロジーズ 2018 Q1               nan             nan              nan   
安藤・間           2008 Q3               nan             nan              nan   
富士ピー・エス        2011 Q4               nan             nan              nan   
廣済堂            2015 Q2               nan             nan              nan   
新明和工業          2008 Q2               nan             nan              nan   
日本ライフライン       2008 Q4               nan             nan              nan   
日本電波工業         2016 Q1               nan             nan              nan   
日精樹脂工業         2010 Q2               nan             nan              nan   
東洋シヤッター        2017 Q1               nan             nan              nan   
高千穂交易          2017 Q1               nan             nan              nan   
ＢＥＥＮＯＳ         2018 Q4               nan             nan              nan   
ＯＵＧホールディングス    2014 Q1               nan             nan              nan   

                         １株当たり税引前利益  
企業名            会計年度 四半期              
ウェルネット         2009 Q1          nan  
               2010 Q2         0.00  
システムソフト        2010 Q1         0.00  
ジャムコ           2008 Q2         0.00  
スターティアホールディングス 2009 Q3         0.00  
ティラド           2009 Q4         0.00  
トリドールホールディングス  2010 Q2         0.00  
ノーリツ鋼機         2011 Q1         0.00  
フォーカスシステムズ     2013 Q1         0.00  
フジオフードグループ本社   2008 Q3          nan  
ペッパーフードサービス    2012 Q1         0.00  
モリテック　スチール     2011 Q4         0.00  
協栄産業           2012 Q3         0.00  
大豊建設           2013 Q1         0.00  
大阪チタニウムテクノロジーズ 2018 Q1         0.00  
安藤・間           2008 Q3         0.00  
富士ピー・エス        2011 Q4         0.00  
廣済堂            2015 Q2         0.00  
新明和工業          2008 Q2         0.00  
日本ライフライン       2008 Q4         0.00  
日本電波工業         2016 Q1         0.00  
日精樹脂工業         2010 Q2         0.00  
東洋シヤッター        2017 Q1         0.00  
高千穂交易          2017 Q1         0.00  
ＢＥＥＮＯＳ         2018 Q4         0.00  
ＯＵＧホールディングス    2014 Q1         0.00

In [43]:
c = collections.Counter(list(df_2008.loc[pd.IndexSlice[:, 2008:, :], [
    '税金等調整前当期純利益［累計］', 
    '税金等調整前当期利益［累計］',
    '【ＱＴＲ】税金等調整前当期利益', 
    '１株当たり税引前利益']][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], '税金等調整前当期純利益［累計］'].isna()
].index.get_level_values(0)))

c.values()

dict_values([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [44]:
c.most_common()

[('ウェルネット', 2),
 ('システムソフト', 1),
 ('ジャムコ', 1),
 ('スターティアホールディングス', 1),
 ('ティラド', 1),
 ('トリドールホールディングス', 1),
 ('ノーリツ鋼機', 1),
 ('フォーカスシステムズ', 1),
 ('フジオフードグループ本社', 1),
 ('ペッパーフードサービス', 1),
 ('モリテック\u3000スチール', 1),
 ('協栄産業', 1),
 ('大豊建設', 1),
 ('大阪チタニウムテクノロジーズ', 1),
 ('安藤・間', 1),
 ('富士ピー・エス', 1),
 ('廣済堂', 1),
 ('新明和工業', 1),
 ('日本ライフライン', 1),
 ('日本電波工業', 1),
 ('日精樹脂工業', 1),
 ('東洋シヤッター', 1),
 ('高千穂交易', 1),
 ('ＢＥＥＮＯＳ', 1),
 ('ＯＵＧホールディングス', 1)]

In [47]:
df_2008.loc[pd.IndexSlice[c.most_common()[0][0], 2008:, :], [
    '税金等調整前当期純利益［累計］', 
    '税金等調整前当期利益［累計］',
    '【ＱＴＲ】税金等調整前当期利益', 
    '１株当たり税引前利益'
]]

税金等調整前当期純利益［累計］   税金等調整前当期利益［累計］  【ＱＴＲ】税金等調整前当期利益  １株当たり税引前利益
企業名    会計年度 四半期                                                               
ウェルネット 2008 Q1    535,000,000.00   535,000,000.00    87,000,000.00    7,005.09
            Q2    200,000,000.00   200,000,000.00   200,000,000.00   10,375.73
            Q3    388,000,000.00   388,000,000.00   188,000,000.00    9,929.37
            Q4    546,000,000.00   546,000,000.00   158,000,000.00    9,274.36
       2009 Q1               nan              nan              nan         nan
            Q2    217,000,000.00   217,000,000.00              nan   11,111.54
            Q3    598,000,000.00   598,000,000.00   381,000,000.00   14,782.22
            Q4  1,175,000,000.00 1,175,000,000.00   577,000,000.00   19,136.50
       2010 Q1  3,540,000,000.00 3,540,000,000.00 2,365,000,000.00         nan
            Q2               nan              nan              nan        0.00
            Q3    300,000,000.00   300,000,000.00              nan    5,979.13
            Q4    455,000,000.00   455,000,000.00   155,000,000.00    6,045.57
       2011 Q1    620,000,000.00   620,000,000.00   165,000,000.00    6,178.44
            Q2    891,000,000.00   891,000,000.00              nan   35,516.05
            Q3  1,172,000,000.00 1,172,000,000.00   281,000,000.00   23,358.48
            Q4  1,621,000,000.00 1,621,000,000.00   449,000,000.00   21,538.17
       2012 Q1  1,788,000,000.00 1,788,000,000.00   167,000,000.00         nan
            Q2    292,000,000.00   292,000,000.00   292,000,000.00      116.39
            Q3    659,000,000.00   659,000,000.00   367,000,000.00      131.32
            Q4    883,000,000.00   883,000,000.00   224,000,000.00      117.30
       2013 Q1  1,237,000,000.00 1,237,000,000.00              nan      123.24
            Q2    419,000,000.00   419,000,000.00   419,000,000.00      166.96
            Q3    806,000,000.00   806,000,000.00   387,000,000.00      162.02
            Q4  1,225,000,000.00 1,225,000,000.00   419,000,000.00      164.74
       2014 Q1  1,466,000,000.00 1,466,000,000.00   241,000,000.00      148.53
            Q2    448,000,000.00   448,000,000.00   448,000,000.00      184.21
            Q3    915,000,000.00   915,000,000.00   467,000,000.00      188.06
            Q4  1,228,000,000.00 1,228,000,000.00   313,000,000.00      168.38
       2015 Q1  1,520,000,000.00 1,520,000,000.00   292,000,000.00      156.74
            Q2    539,000,000.00   539,000,000.00   539,000,000.00      225.70
            Q3  1,110,000,000.00 1,110,000,000.00   571,000,000.00      234.37
            Q4  1,712,000,000.00 1,712,000,000.00   602,000,000.00      242.34
       2016 Q1  2,007,000,000.00 2,007,000,000.00   295,000,000.00      205.85
            Q2    612,000,000.00   612,000,000.00   612,000,000.00      131.21
            Q3    930,000,000.00   930,000,000.00   318,000,000.00       99.49
            Q4  1,153,000,000.00 1,153,000,000.00   223,000,000.00       82.11
       2017 Q1  1,239,000,000.00 1,239,000,000.00    86,000,000.00       66.05
            Q2    229,000,000.00   229,000,000.00   229,000,000.00       48.48
            Q3    366,000,000.00   366,000,000.00   137,000,000.00       38.71
            Q4    601,000,000.00   601,000,000.00   235,000,000.00       42.41
       2018 Q1    708,000,000.00   708,000,000.00   107,000,000.00       37.62
            Q2    149,000,000.00   149,000,000.00   149,000,000.00       31.94
            Q3    306,000,000.00   306,000,000.00   157,000,000.00       32.72
            Q4    470,000,000.00   470,000,000.00   164,000,000.00       33.47
       2019 Q1    537,000,000.00   537,000,000.00    67,000,000.00       28.67
            Q2    231,000,000.00   231,000,000.00   231,000,000.00       49.20
            Q3    485,000,000.00   485,000,000.00   254,000,000.00       51.64
            Q4    710,000,000.00   710,000,000.00   225,000,000.00       50.38
       2020 Q1    718,000,000.00   718,000,000.00     8,000,000.00 

In [ ]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], ['税金等調整前当期純利益［累計］',  '１株当たり税引前利益']][
    (df_2008.loc[pd.IndexSlice[:, 2008:, :], '税金等調整前当期純利益［累計］'].isna()) & 
    (df_2008.loc[pd.IndexSlice[:, 2008:, :],  '１株当たり税引前利益'].notna())
]

税引前利益はETRの計算の分母になるので、0はやばい。dropするかも

* 売上高

In [49]:
# 欠損率
df_2008.loc[pd.IndexSlice[:, 2008:, :], ['売上高・営業収益［累計］', '【ＱＴＲ】売上高・営業収益', '１株当たり売上高']].isna().mean()

売上高・営業収益［累計］    0.00
【ＱＴＲ】売上高・営業収益   0.01
１株当たり売上高        0.01
dtype: float64

In [53]:
# 欠損状況
df_2008.loc[pd.IndexSlice[:, 2008:, :], ['売上高・営業収益［累計］', '【ＱＴＲ】売上高・営業収益', '１株当たり売上高']][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], '売上高・営業収益［累計］'].isna()
]

売上高・営業収益［累計］  【ＱＴＲ】売上高・営業収益  １株当たり売上高
企業名           会計年度 四半期                                       
ウェルネット        2009 Q1            nan            nan       nan
トリドールホールディングス 2010 Q2            nan            nan       nan
フジオフードグループ本社  2008 Q3            nan            nan       nan
モリテック　スチール    2011 Q4            nan            nan       nan
三井物産          2016 Q1            nan            nan       nan
...                              ...            ...       ...
豊田通商          2019 Q4            nan            nan       nan
              2020 Q1            nan            nan       nan
                   Q2            nan            nan       nan
                   Q3            nan            nan       nan
                   Q4            nan            nan       nan

[168 rows x 3 columns]

"売上高・営業収益［累計］"の欠損は他のカラムでも欠損しているので補填不可能

In [54]:
c = collections.Counter(list(df_2008.loc[pd.IndexSlice[:, 2008:, :], '売上高・営業収益［累計］'][
    df_2008.loc[pd.IndexSlice[:, 2008:, :], '売上高・営業収益［累計］'].isna()
].index.get_level_values(0)))

c.values()

dict_values([1, 1, 1, 1, 20, 33, 16, 24, 20, 16, 12, 1, 1, 21])

In [55]:
c.most_common()

[('三菱商事', 33),
 ('伊藤忠商事', 24),
 ('豊田通商', 21),
 ('三井物産', 20),
 ('住友商事', 20),
 ('丸紅', 16),
 ('兼松', 16),
 ('双日', 12),
 ('ウェルネット', 1),
 ('トリドールホールディングス', 1),
 ('フジオフードグループ本社', 1),
 ('モリテック\u3000スチール', 1),
 ('富士ピー・エス', 1),
 ('日本ライフライン', 1)]

In [65]:
df_2008.loc[pd.IndexSlice[c.most_common()[7][0], 2008:, :], ['売上高・営業収益［累計］', '【ＱＴＲ】売上高・営業収益', '１株当たり売上高']]

売上高・営業収益［累計］        【ＱＴＲ】売上高・営業収益  １株当たり売上高
企業名 会計年度 四半期                                                    
双日  2008 Q1  1,407,178,000,000.00 1,407,178,000,000.00  4,562.98
         Q2  2,905,109,000,000.00 1,497,931,000,000.00  4,710.14
         Q3  4,210,238,000,000.00 1,305,129,000,000.00  4,550.87
         Q4  5,166,182,000,000.00   955,944,000,000.00  4,188.18
    2009 Q1    897,121,000,000.00   897,121,000,000.00  2,909.29
         Q2  1,886,398,000,000.00   989,277,000,000.00  3,058.73
         Q3  2,869,018,000,000.00   982,620,000,000.00  3,089.77
         Q4  3,844,418,000,000.00   975,400,000,000.00  3,097.14
    2010 Q1    958,289,000,000.00   958,289,000,000.00  3,063.85
         Q2  1,965,179,000,000.00 1,006,890,000,000.00  3,141.55
         Q3  2,958,905,000,000.00   993,726,000,000.00  3,153.42
         Q4  4,014,639,000,000.00 1,055,734,000,000.00  3,208.92
    2011 Q1  1,109,645,000,000.00 1,109,645,000,000.00  3,547.90
         Q2  2,195,704,000,000.00 1,086,059,000,000.00  3,510.22
         Q3  3,300,577,000,000.00 1,104,873,000,000.00  3,517.71
         Q4  4,321,734,000,000.00                  nan  3,454.36
    2012 Q1  1,010,607,000,000.00 1,010,607,000,000.00       nan
         Q2  1,959,827,000,000.00   949,220,000,000.00       nan
         Q3  2,952,520,000,000.00   992,693,000,000.00       nan
         Q4  3,934,456,000,000.00   981,936,000,000.00  3,144.83
    2013 Q1  1,007,422,000,000.00 1,007,422,000,000.00  3,220.97
         Q2  2,002,158,000,000.00   994,736,000,000.00  3,200.69
         Q3  3,062,228,000,000.00 1,060,070,000,000.00  3,263.56
         Q4  4,046,577,000,000.00   984,349,000,000.00  3,234.50
    2014 Q1    946,862,000,000.00   946,862,000,000.00  3,027.46
         Q2  1,944,836,000,000.00   997,974,000,000.00  3,109.18
         Q3  3,013,564,000,000.00 1,068,728,000,000.00  3,211.83
         Q4  4,105,295,000,000.00 1,091,731,000,000.00  3,281.54
    2015 Q1  1,030,091,000,000.00 1,030,091,000,000.00  3,293.60
         Q2  2,015,907,000,000.00   985,816,000,000.00  3,222.82
         Q3  3,024,447,000,000.00 1,008,540,000,000.00  3,223.45
         Q4  4,006,649,000,000.00   982,202,000,000.00  3,202.71
    2016 Q1    896,291,000,000.00   896,291,000,000.00  2,865.81
         Q2  1,776,674,000,000.00   880,383,000,000.00  2,840.38
         Q3  2,749,296,000,000.00   972,622,000,000.00  2,930.21
         Q4  3,745,549,000,000.00   996,253,000,000.00  2,994.02
    2017 Q1  1,000,697,000,000.00 1,000,697,000,000.00  3,199.72
         Q2  2,044,315,000,000.00 1,043,618,000,000.00  3,268.35
         Q3  3,139,354,000,000.00 1,095,039,000,000.00  3,346.03
         Q4  4,209,077,000,000.00 1,069,723,000,000.00  3,364.64
    2018 Q1                   nan                  nan       nan
         Q2                   nan                  nan       nan
         Q3                   nan                  nan       nan
         Q4                   nan                  nan       nan
    2019 Q1                   nan                  nan       nan
         Q2                   nan                  nan       nan
         Q3                   nan                  nan       nan
         Q4                   nan                  nan       nan
    2020 Q1                   nan                  nan       nan
         Q2                   nan                  nan       nan
         Q3                   nan                  nan       nan
         Q4                   nan                  nan       nan

特定の企業に欠損が集中 (drop)

In [67]:
df_2008.columns

Index(['決算期（年）', '決算期（月）', '決算期', '連結基準フラグ', '決算種別フラグ', '決算発表日',
       '事業年度開始年月日［３ヵ月］', '事業年度終了年月日', '１株当たり利益［３ヵ月］', '１株当たり利益［累計］',
       '当期純利益（連結）［累計］', '【ＱＴＲ】当期利益', '棚卸資産', '受取手形・売掛金／売掛金及びその他の短期債権', '資本的支出',
       '売上総利益［累計］', '【ＱＴＲ】売上総利益', '販売費及び一般管理費［累計］', '法人税等［累計］',
       '法人税・住民税及び事業税合計／当期税金費用［累計］', '税金等調整前当期純利益［累計］', '税金等調整前当期利益［累計］',
       '【ＱＴＲ】税金等調整前当期利益', '１株当たり税引前利益', '売上高・営業収益［累計］', '【ＱＴＲ】売上高・営業収益',
       '１株当たり売上高', '期末従業員数', '純資産合計／資本合計', '【ＱＴＲ】純資産', '１株当たり純資産',
       '期中平均株式数［累計］', '期中平均株式数［３ヵ月］', '期末発行済株式総数', '自己株式数'],
      dtype='object')

欠損処理後、最後に期間を2008Q1~にカットする

## ファンダメンタル会計変数の生成

## Data Preprocessing

データのスケーリング

$$\begin{eqnarray}
    INV &=& \frac{棚卸資産}{普通株式の期中加重平均株式数} \\
    AR &=& \frac{売掛金}{普通株式の期中加重平均株式数} \\
    CAPX &=& \frac{資本的支出}{普通株式の期中加重平均株式数} \\
    GM &=& \frac{売上総利益}{普通株式の期中加重平均株式数} \\
    SA &=& \frac{販売費及び一般管理費}{普通株式の期中加重平均株式数} \\
    ETR &=& 実効税率 \\
    LF &=& \log{労働力} \\
\end{eqnarray}$$